## Início do Pipeline de Modelagem e Rastreamento com MLflow

Este notebook tem como objetivo iniciar o pipeline de experimentos, carregando os datasets finais da camada `Curated` para criar os conjuntos de treino e teste totalmente alinhados. O fluxo inclui a separação de variáveis preditoras (`X`) e variável alvo (`y`), além da configuração inicial do MLflow Tracking, garantindo que todos os parâmetros, métricas e artefatos do modelo sejam rastreados de forma coerente e versionável.


In [2]:
# 🔧 ETAPA: FIXAR CWD EM /workspace PARA CONSISTÊNCIA GLOBAL

import os

CWD_FIXO = "/workspace"
os.chdir(CWD_FIXO)

print("✅ Diretório de trabalho fixado em:", os.getcwd())


✅ Diretório de trabalho fixado em: /workspace


## Experimento Baseline com MLflow e Monitoramento de Progresso

Nesta etapa será rodado o primeiro experimento baseline usando o MLflow para rastrear parâmetros, métricas e artefatos. Para acompanhar operações potencialmente demoradas, como o ajuste do modelo (`fit`) e a geração de métricas, será utilizado o `tqdm` para monitorar loops de forma explícita. Isso garante visibilidade do progresso em tempo real, além de manter a rastreabilidade completa do pipeline.


In [3]:
# 🔧 ETAPA: CARGA DOS DADOS CURADOS E CONFIGURAÇÃO DO MLFLOW TRACKING

"""
Executa:
1) Fixação obrigatória do diretório de trabalho como /workspace
2) Carregamento de 'train_curated.csv' e 'test_curated.csv'
3) Separação de X_train, y_train, X_test, y_test
4) Configuração do Tracking URI do MLflow para apontar ao serviço interno
"""

import os
import pandas as pd
import mlflow

# 1️⃣ Fixar CWD
CWD_FIXO = "/workspace"
os.chdir(CWD_FIXO)
print("✅ Diretório de trabalho fixado em:", os.getcwd())

# 2️⃣ Definir paths absolutos coerentes
TRAIN_PATH = os.path.join("data", "curated", "train_curated.csv")
TEST_PATH = os.path.join("data", "curated", "test_curated.csv")

# 3️⃣ Carregar datasets curados
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print("\n✅ Treino shape:", train_df.shape)
print("✅ Teste shape :", test_df.shape)

# 4️⃣ Separar variáveis explicativas e target
TARGET = "Credit_Score_Standard"  # Altere aqui se o target for outro

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

X_test = test_df.drop(columns=[TARGET])
y_test = test_df[TARGET]

print("\n✅ X_train:", X_train.shape)
print("✅ y_train:", y_train.shape)
print("✅ X_test :", X_test.shape)
print("✅ y_test :", y_test.shape)

# 5️⃣ Configurar MLflow Tracking URI interno (dentro do container)
mlflow.set_tracking_uri("http://mlflow:5000")
print("\n✅ MLflow Tracking URI configurado para:", mlflow.get_tracking_uri())


✅ Diretório de trabalho fixado em: /workspace

✅ Treino shape: (100000, 6305)
✅ Teste shape : (50000, 6305)

✅ X_train: (100000, 6304)
✅ y_train: (100000,)
✅ X_test : (50000, 6304)
✅ y_test : (50000,)

✅ MLflow Tracking URI configurado para: http://mlflow:5000


In [ ]:
#  ETAPA: Refazer Baseline — Normalização do Working Directory, Carga Curated, Reconstrução de y e Tracking MLflow

"""
Executa:
1) Validação e normalização do diretório de trabalho (CWD).
2) Carregamento de 'train_curated.csv' e 'test_curated.csv'.
3) Reconstrução de y a partir de colunas dummy.
4) Separação coerente de X e y.
5) Tracking URI e credenciais MinIO explícitos.
6) Treino DecisionTreeClassifier com barra de progresso.
7) Logging MLflow de hiperparâmetros, métricas e artefato.
8) Prints finais coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Validar e corrigir CWD
print("Current Working Directory (antes):", os.getcwd())
os.chdir('/workspace')
print("Current Working Directory (depois):", os.getcwd())

# 2️⃣ Paths absolutos coerentes
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carregar datasets
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print("\nTreino shape:", train_df.shape)
print("Teste shape:", test_df.shape)
print("\ntrain_df.head(5):")
print(train_df.head(5))

# 4️⃣ Reconstruir y a partir de colunas dummy
dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("\nColunas de classe detectadas:", dummy_cols)

# Reconstrói y_train
y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

# Reconstrói y_test
y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# 5️⃣ Tracking URI e credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

print("\nTracking URI:", mlflow.get_tracking_uri())
print("MLFLOW_S3_ENDPOINT_URL:", os.environ['MLFLOW_S3_ENDPOINT_URL'])

# 6️⃣ Cria/recupera experimento
experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="Baseline_DecisionTree_Curated") as run:
    params = {"max_depth": 5, "random_state": 42}
    mlflow.log_params(params)

    model = DecisionTreeClassifier(**params)

    print("\nTreinando modelo com barra de progresso:")
    for _ in tqdm(range(1), desc="Fitting model"):
        model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nAccuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")

    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_score", f1)
    mlflow.sklearn.log_model(model, "model")

    # ✅ Prints finais coerentes — SOMENTE com 127.0.0.1
    print(f"\nRun ID: {run.info.run_id}")
    print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")
    print(f"View run Baseline_DecisionTree_Curated at: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")
    print(f"View experiment at: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}")


Current Working Directory (antes): /workspace/notebooks
Current Working Directory (depois): /workspace

Treino shape: (100000, 6305)
Teste shape: (50000, 6305)

train_df.head(5):
     Age  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  \
0   23.0       19114.12            1824.843333                  3   
1   23.0       19114.12                    NaN                  3   
2 -500.0       19114.12                    NaN                  3   
3   23.0       19114.12                    NaN                  3   
4   23.0       19114.12            1824.843333                  3   

   Num_Credit_Card  Interest_Rate  Delay_from_due_date  Num_Credit_Inquiries  \
0                4              3                    3                   4.0   
1                4              3                   -1                   4.0   
2                4              3                    3                   4.0   
3                4              3                    5                   4.0   
4     

Fitting model: 100%|██████████| 1/1 [00:12<00:00, 12.90s/it]



Accuracy: 0.5496
F1 Score: 0.7093

Run ID: 86de72fd063e485fa584c1d8c0395aca
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
View run Baseline_DecisionTree_Curated at: http://127.0.0.1:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run Baseline_DecisionTree_Curated at: http://mlflow:5000/#/experiments/1/runs/86de72fd063e485fa584c1d8c0395aca
🧪 View experiment at: http://mlflow:5000/#/experiments/1


## ETAPA: Melhoria do Modelo com GridSearchCV e MLflow Tracking

Este bloco marca a transição do experimento baseline para uma etapa de otimização incremental, usando GridSearchCV para explorar múltiplas combinações de hiperparâmetros de forma sistemática e rastreada.

## Objetivo
- Encontrar a configuração de hiperparâmetros mais eficaz para a Árvore de Decisão (DecisionTreeClassifier).
- Registrar cada combinação testada como um run único no MLflow, com seus parâmetros, métricas e artefato final.

## Princípios aplicados
- Tracking URI e backend MinIO/S3 mantidos consistentes.
- Cada variação é rastreável, sem sobrescrever runs anteriores.
- Uso de tqdm para barra de progresso, garantindo visibilidade em loops demorados.
- Prints finais coerentes, com links 127.0.0.1 para acesso ao MLflow UI.

## Resultado esperado
- Métricas comparáveis entre baseline e grid search.
- Melhor modelo salvo como artefato no bucket MinIO.
- Próximo passo: preparar o pipeline para registrar o modelo validado no Registry do MLflow.


In [ ]:
# 🔧 ETAPA: GridSearch Manual com ParameterGrid, tqdm e Tracking MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega train_curated e test_curated.
3) Reconstrói y a partir das colunas dummy.
4) Itera ParameterGrid manualmente.
5) Barra tqdm avança por combinação.
6) Loga cada combinação como run separado no MLflow.
7) Prints coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carga + reconstrução y
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("Colunas classe:", dummy_cols)

y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")

# 4️⃣ ParameterGrid manual
param_grid = {
    "max_depth": [3, 5, 7],
    "min_samples_split": [2, 5, 10]
}
grid = ParameterGrid(param_grid)

# 5️⃣ Tracking URI + credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual...")

for params in tqdm(grid, desc="Runs"):
    with mlflow.start_run(run_name=f"GridSearch_Manual_{params}") as run:
        model = DecisionTreeClassifier(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace
CWD depois: /workspace
Colunas classe: ['Credit_Score_Poor', 'Credit_Score_Standard']
X_train: (100000, 6303) | y_train: (100000,)

Executando GridSearch manual...


Runs:  11%|█         | 1/9 [00:15<02:06, 15.75s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2}
Accuracy: 0.5388 | F1 Score: 0.7003
Run ID: 8347a501625844da89aaa891510faa55
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/8347a501625844da89aaa891510faa55
🏃 View run GridSearch_Manual_{'max_depth': 3, 'min_samples_split': 2} at: http://mlflow:5000/#/experiments/1/runs/8347a501625844da89aaa891510faa55
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs:  22%|██▏       | 2/9 [00:27<01:34, 13.56s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 5}
Accuracy: 0.5388 | F1 Score: 0.7003
Run ID: 9f9ded00e4594c7197535ae840bc8bfc
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/9f9ded00e4594c7197535ae840bc8bfc
🏃 View run GridSearch_Manual_{'max_depth': 3, 'min_samples_split': 5} at: http://mlflow:5000/#/experiments/1/runs/9f9ded00e4594c7197535ae840bc8bfc
🧪 View experiment at: http://mlflow:5000/#/experiments/1


...


Combinação: {'max_depth': 7, 'min_samples_split': 2}
Accuracy: 0.5422 | F1 Score: 0.7032
Run ID: 9c9d83b3de294b5aab2357b42a67aa06
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/9c9d83b3de294b5aab2357b42a67aa06
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 2} at: http://mlflow:5000/#/experiments/1/runs/9c9d83b3de294b5aab2357b42a67aa06
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs:  89%|████████▉ | 8/9 [01:35<00:11, 11.69s/it]


Combinação: {'max_depth': 7, 'min_samples_split': 5}
Accuracy: 0.5422 | F1 Score: 0.7032
Run ID: d2685d32bdcb42b783e358095e9f5b63
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/d2685d32bdcb42b783e358095e9f5b63
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 5} at: http://mlflow:5000/#/experiments/1/runs/d2685d32bdcb42b783e358095e9f5b63
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs: 100%|██████████| 9/9 [01:46<00:00, 11.88s/it]


Combinação: {'max_depth': 7, 'min_samples_split': 10}
Accuracy: 0.5424 | F1 Score: 0.7033
Run ID: 0cfc26f47de549969c8fdb2ceef4565c
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/0cfc26f47de549969c8fdb2ceef4565c
🏃 View run GridSearch_Manual_{'max_depth': 7, 'min_samples_split': 10} at: http://mlflow:5000/#/experiments/1/runs/0cfc26f47de549969c8fdb2ceef4565c
🧪 View experiment at: http://mlflow:5000/#/experiments/1


In [ ]:
# 🔧 ETAPA: GridSearch Manual com RandomForest, ParameterGrid, tqdm e Tracking MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega train_curated e test_curated.
3) Reconstrói y a partir das colunas dummy.
4) Executa GridSearch manual com RandomForestClassifier.
5) Usa ParameterGrid + tqdm para barra real por combinação.
6) Loga cada run no MLflow com credenciais MinIO coerentes.
7) Prints finais coerentes com links 127.0.0.1.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante do MLflow
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Paths
TRAIN_PATH = 'data/curated/train_curated.csv'
TEST_PATH = 'data/curated/test_curated.csv'

# 3️⃣ Carga + reconstrução y
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

dummy_cols = [col for col in train_df.columns if col.startswith('Credit_Score_')]
print("Colunas classe:", dummy_cols)

y_train = train_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_train = train_df.drop(columns=dummy_cols)

y_test = test_df[dummy_cols].idxmax(axis=1).str.replace('Credit_Score_', '')
X_test = test_df.drop(columns=dummy_cols)

print(f"X_train: {X_train.shape} | y_train: {y_train.shape}")

# 4️⃣ ParameterGrid com RandomForest
param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5],
    "min_samples_split": [2, 5]
}
grid = ParameterGrid(param_grid)

# 5️⃣ Tracking URI + credenciais MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual RandomForest...")

for params in tqdm(grid, desc="Runs RandomForest"):
    with mlflow.start_run(run_name=f"GridSearch_RF_{params}") as run:
        model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace
CWD depois: /workspace
Colunas classe: ['Credit_Score_Poor', 'Credit_Score_Standard']
X_train: (100000, 6303) | y_train: (100000,)

Executando GridSearch manual RandomForest...


Runs RandomForest:  12%|█▎        | 1/8 [00:10<01:14, 10.66s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50}
Accuracy: 0.1160 | F1 Score: 0.2078
Run ID: 7e1e8f44f2a14b469b513b183d4a0ae5
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/7e1e8f44f2a14b469b513b183d4a0ae5
🏃 View run GridSearch_RF_{'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 50} at: http://mlflow:5000/#/experiments/1/runs/7e1e8f44f2a14b469b513b183d4a0ae5
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest:  25%|██▌       | 2/8 [00:18<00:55,  9.26s/it]


Combinação: {'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.0560 | F1 Score: 0.1061
Run ID: 7e2a7fd6532a415790f7e0297eaef518
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/7e2a7fd6532a415790f7e0297eaef518
🏃 View run GridSearch_RF_{'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/7e2a7fd6532a415790f7e0297eaef518
🧪 View experiment at: http://mlflow:5000/#/experiments/1


...


Combinação: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.1582 | F1 Score: 0.2732
Run ID: 3a35386b08854259abb939bffb29abb8
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/3a35386b08854259abb939bffb29abb8
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/3a35386b08854259abb939bffb29abb8
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest:  88%|████████▊ | 7/8 [00:54<00:07,  7.32s/it]


Combinação: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50}
Accuracy: 0.2117 | F1 Score: 0.3494
Run ID: ee503b2c1af948db890c83f7a178a5de
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/ee503b2c1af948db890c83f7a178a5de
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50} at: http://mlflow:5000/#/experiments/1/runs/ee503b2c1af948db890c83f7a178a5de
🧪 View experiment at: http://mlflow:5000/#/experiments/1


Runs RandomForest: 100%|██████████| 8/8 [01:02<00:00,  7.78s/it]


Combinação: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy: 0.1581 | F1 Score: 0.2730
Run ID: 4693946ae8bf47a799823ab6e1844b93
Acesse: http://127.0.0.1:5000/#/experiments/1/runs/4693946ae8bf47a799823ab6e1844b93
🏃 View run GridSearch_RF_{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100} at: http://mlflow:5000/#/experiments/1/runs/4693946ae8bf47a799823ab6e1844b93
🧪 View experiment at: http://mlflow:5000/#/experiments/1


In [ ]:
# 🔧 ETAPA: GridSearch Manual com GradientBoostingClassifier, ParameterGrid, tqdm e MLflow

"""
Executa:
1) Normaliza o CWD para '/workspace'.
2) Carrega 'train_clean.csv' da camada processed.
3) Separa X e y com target original.
4) Aplica pd.get_dummies() no X.
5) Usa train_test_split com stratify.
6) Executa GridSearch manual com GradientBoostingClassifier.
7) Barra tqdm para progresso real.
8) Loga cada run separadamente no MLflow.
"""

import os
import logging
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ Silenciar logger redundante
logging.getLogger("mlflow").setLevel(logging.ERROR)

# 1️⃣ Normalizar CWD
print("CWD antes:", os.getcwd())
os.chdir('/workspace')
print("CWD depois:", os.getcwd())

# 2️⃣ Carregar dados
df = pd.read_csv('data/processed/train_clean.csv')
print("\ndf shape:", df.shape)
print("\ndf.head(5):\n", df.head(5))

# 3️⃣ Separa X e y
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

# 4️⃣ Pré-processa X igual curated
X = pd.get_dummies(X)

# 5️⃣ train_test_split + alinhamento coerente
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print(f"\nX_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

# 6️⃣ Configurar ParameterGrid
param_grid = {
    "n_estimators": [50, 100],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1]
}
grid = ParameterGrid(param_grid)

# 7️⃣ Tracking MLflow + MinIO
mlflow.set_tracking_uri("http://mlflow:5000")
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'

experiment_name = "QuantumFinance_CreditScore"
mlflow.set_experiment(experiment_name)

print("\nExecutando GridSearch manual — GradientBoostingClassifier...")

for params in tqdm(grid, desc="Runs GBoost"):
    with mlflow.start_run(run_name=f"GridSearch_GBoost_{params}") as run:
        model = GradientBoostingClassifier(**params, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.sklearn.log_model(model, "model")

        print(f"\nCombinação: {params}")
        print(f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
        print(f"Run ID: {run.info.run_id}")
        print(f"Acesse: http://127.0.0.1:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


CWD antes: /workspace/notebooks
CWD depois: /workspace

df shape: (100000, 28)

df.head(5):
        ID Customer_ID     Month           Name    Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh   23.0  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh   23.0  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh -500.0  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh   23.0  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh   23.0  821-00-0265  Scientist   

   Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0       19114.12            1824.843333                  3  ...     Unknown   
1       19114.12                    NaN                  3  ...        Good   
2       19114.12                    NaN                  3  ...        Good   
3       19114.12                    NaN                  3  ...        Good   
4       19114.12         

: 

#  Diagnóstico do Footprint de Memória

Antes de rodar fitting, esta célula calcula o uso real de memória dos DataFrames `X_train` e `y_train`.  
Garante rastreabilidade sobre quanta RAM o kernel irá consumir, considerando `deep=True` para contar objetos, ponteiros e índices.  
Este valor deve ser menor que 70% da RAM real do container para evitar travamento por OOM Killer.


## Recarga dos Datasets Curated — Caminho corrigido

Esta célula recarrega os datasets `train_curated.csv` e `test_curated.csv` usando caminho relativo correto, garantindo coerência com a estrutura `/workspace/`.



In [ ]:
# ETAPA: Recarga dos Datasets Curated

import pandas as pd

train_df = pd.read_csv("../data/curated/train_curated.csv")
test_df = pd.read_csv("../data/curated/test_curated.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")


Train shape: (100000, 6305)
Test shape: (50000, 6305)


In [ ]:
print(train_df.columns.tolist())


['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance', 'Num_of_Loan_Bin', 'Changed_Credit_Limit_Bin', 'Num_of_Delayed_Payment_Bin', 'Credit_History_Age_Bin', 'Month_Num', 'Occupation_Architect', 'Occupation_Developer', 'Occupation_Doctor', 'Occupation_Engineer', 'Occupation_Entrepreneur', 'Occupation_Journalist', 'Occupation_Lawyer', 'Occupation_Manager', 'Occupation_Mechanic', 'Occupation_Media_Manager', 'Occupation_Musician', 'Occupation_Scientist', 'Occupation_Teacher', 'Occupation_Unknown', 'Occupation_Writer', 'Type_of_Loan_Auto Loan, Auto Loan, Auto Loan, Auto Loan, Credit-Builder Loan, Credit-Builder Loan, Mortgage Loan, and Personal Loan', 'Type_of_Loan_Auto Loan, Auto Loan, Auto Loan, Auto Loan, Student Loan, and Student Loan', 'Type_of_Loan_Auto Loan, Auto Loan, A

---
REABRIR O FEATURE_ENGINEERING_CURADORIA.IPNYB PARA DIMINUIR CARDINALIDADE
---

---


## Extensão Controlada — One-Hot Encoding Restrito Pré-Cardinalidade

Este bloco aplica o **One-Hot Encoding restrito** nas variáveis `Month`, `Occupation_Group` e `Payment_Behaviour`, antes de qualquer novo diagnóstico de cardinalidade.

O objetivo é observar quantas colunas serão adicionadas e comparar com o pipeline anterior (+6.300 colunas) para verificar se o footprint segue controlado.
O resultado será salvo como `CURATED V1.1` para rastreabilidade total.


In [ ]:
# ETAPA: ONE-HOT ENCODING RESTRITO V1.1 E COMPARACAO IMEDIATA

import pandas as pd

# Caminhos
train_curated_v1 = '/workspace/data/curated/train_curated_v1.csv'
test_curated_v1  = '/workspace/data/curated/test_curated_v1.csv'

# Carrega
train_df = pd.read_csv(train_curated_v1)
test_df  = pd.read_csv(test_curated_v1)

# Colunas a codificar
cols_to_encode = ['Month', 'Occupation_Group', 'Payment_Behaviour']

# Aplica OHE restrito
train_encoded = pd.get_dummies(train_df, columns=cols_to_encode, drop_first=True)
test_encoded  = pd.get_dummies(test_df, columns=cols_to_encode, drop_first=True)

# Alinha colunas para garantir mesma estrutura
train_encoded, test_encoded = train_encoded.align(test_encoded, join='outer', axis=1, fill_value=0)

# Compara shape
print("\nShape original V1 (train):", train_df.shape)
print("Shape V1.1 após OHE (train):", train_encoded.shape)

print("\nShape original V1 (test):", test_df.shape)
print("Shape V1.1 após OHE (test):", test_encoded.shape)

# Salva V1.1
train_encoded.to_csv('/workspace/data/curated/train_curated_v1_1.csv', index=False)
test_encoded.to_csv('/workspace/data/curated/test_curated_v1_1.csv', index=False)

print("\nSnapshots CURATED V1.1 salvos.")



Shape original V1 (train): (100000, 65)
Shape V1.1 após OHE (train): (100000, 93)

Shape original V1 (test): (50000, 64)
Shape V1.1 após OHE (test): (50000, 93)

Snapshots CURATED V1.1 salvos.


## Versionamento Atômico — Snapshot CURATED V1.1

Este bloco faz o versionamento atômico do `train_curated_v1_1.csv` e `test_curated_v1_1.csv` com `DVC` e `Git`.  
O fluxo garante rastreabilidade total: verificação física, commit coerente, push para backend MinIO.


In [ ]:
# ETAPA: VERSIONAMENTO ATÔMICO CURATED V1.1

import os
import subprocess

# Caminhos V1.1
train_curated_v1_1 = '/workspace/data/curated/train_curated_v1_1.csv'
test_curated_v1_1  = '/workspace/data/curated/test_curated_v1_1.csv'

# Verifica CWD
print("\nDiretório de trabalho atual:", os.getcwd())

# Confirma existência física
print("\nVerificando existência física:")
print("TRAIN V1.1:", os.path.exists(train_curated_v1_1))
print("TEST V1.1 :", os.path.exists(test_curated_v1_1))

if not os.path.exists(train_curated_v1_1) or not os.path.exists(test_curated_v1_1):
    raise FileNotFoundError("Um dos arquivos CURATED V1.1 não foi encontrado.")

# DVC add
print("\nExecutando dvc add ...")
subprocess.run(['dvc', 'add', train_curated_v1_1], check=True)
subprocess.run(['dvc', 'add', test_curated_v1_1], check=True)

# Git add dos metadados .dvc
print("\nAdicionando metadados .dvc ao Git ...")
subprocess.run(['git', 'add', f"{train_curated_v1_1}.dvc"], check=True)
subprocess.run(['git', 'add', f"{test_curated_v1_1}.dvc"], check=True)

# Commit coerente
print("\nRealizando commit Git ...")
subprocess.run(['git', 'commit', '-m', 'Versionamento CURATED V1.1 com OHE restrito'], check=True)

# DVC push
print("\nExecutando dvc push ...")
subprocess.run(['dvc', 'push'], check=True)

# Git push final
print("\nExecutando git push ...")
subprocess.run(['git', 'push'], check=True)

print("\nVersionamento CURATED V1.1 concluído com sucesso.")



Diretório de trabalho atual: /workspace/notebooks

Verificando existência física:
TRAIN V1.1: True
TEST V1.1 : True

Executando dvc add ...


⠋ Checking graph
⠋ Checking graph



Adicionando metadados .dvc ao Git ...

Realizando commit Git ...
[main 7d92073] Versionamento CURATED V1.1 com OHE restrito
 2 files changed, 10 insertions(+)
 create mode 100644 data/curated/test_curated_v1_1.csv.dvc
 create mode 100644 data/curated/train_curated_v1_1.csv.dvc

Executando dvc push ...
2 files pushed

Executando git push ...

Versionamento CURATED V1.1 concluído com sucesso.


To github.com:WRMELO/MBA_MLOPS.git
   f76be84..7d92073  main -> main


## Diagnóstico de Footprint — Snapshot CURATED V1.1

Este bloco confirma o footprint em **disco** e **RAM** dos arquivos `CURATED V1.1` para manter padrão comparável ao `V1` e ao pipeline original.

Os valores serão exibidos em MB.


In [ ]:
# ETAPA: DIAGNOSTICO FOOTPRINT CURATED V1.1

import os
import pandas as pd

# Funções
def file_size(path):
    size_bytes = os.path.getsize(path)
    return round(size_bytes / (1024 * 1024), 2)

def memory_usage_df(path):
    df = pd.read_csv(path)
    return round(df.memory_usage(deep=True).sum() / (1024 * 1024), 2)

# Caminhos V1.1
train_v1_1 = '/workspace/data/curated/train_curated_v1_1.csv'
test_v1_1  = '/workspace/data/curated/test_curated_v1_1.csv'

# Disco
train_disk = file_size(train_v1_1)
test_disk  = file_size(test_v1_1)

# RAM
train_mem = memory_usage_df(train_v1_1)
test_mem  = memory_usage_df(test_v1_1)

print("\nTamanho em DISCO (MB):")
print(f"TRAIN V1.1: {train_disk} MB")
print(f"TEST V1.1 : {test_disk} MB")

print("\nFootprint em MEMÓRIA (MB):")
print(f"TRAIN V1.1: {train_mem} MB")
print(f"TEST V1.1 : {test_mem} MB")



Tamanho em DISCO (MB):
TRAIN V1.1: 63.33 MB
TEST V1.1 : 30.67 MB

Footprint em MEMÓRIA (MB):
TRAIN V1.1: 165.26 MB
TEST V1.1 : 81.38 MB


In [ ]:
# ETAPA: RECARREGAMENTO E COMPARACAO SHAPE FINAL CURATED V1.1

import pandas as pd

# Caminhos coerentes V1.1
train_curated_v1_1 = '/workspace/data/curated/train_curated_v1_1.csv'
test_curated_v1_1  = '/workspace/data/curated/test_curated_v1_1.csv'

# Recarrega DataFrames
train_df_v1_1 = pd.read_csv(train_curated_v1_1)
test_df_v1_1  = pd.read_csv(test_curated_v1_1)

# Exibe shapes
print("\nShape atual do TRAIN CURATED V1.1:", train_df_v1_1.shape)
print("Shape atual do TEST CURATED V1.1 :", test_df_v1_1.shape)



Shape atual do TRAIN CURATED V1.1: (100000, 93)
Shape atual do TEST CURATED V1.1 : (50000, 93)


## Comparativo de Footprint — Antes, CURATED V1 e CURATED V1.1

Antes da aplicação do binning supervisionado e agrupamentos controlados, o pipeline de Feature Engineering gerava um conjunto com **altíssima cardinalidade**, alcançando **6.305 colunas** por amostra.

- **Train shape (antigo)**: 100.000 linhas × 6.305 colunas  
- **Test shape (antigo)**: 50.000 linhas × 6.305 colunas

Esse volume extremo era causado por **one-hot indiscriminado** em categorias raras e variáveis contínuas pulverizadas, levando a estouros de memória (OOM Killer) mesmo em máquinas robustas.

Após a revisão completa, com:
- Diagnóstico estatístico detalhado,
- Binning supervisionado com faixas coerentes ao negócio,
- Agrupamento de categorias raras,
- Eliminação de redundâncias mantendo rastreabilidade,

o footprint caiu drasticamente para:

- **Train shape (CURATED V1)**: 100.000 linhas × 65 colunas  
- **Test shape (CURATED V1)**: 50.000 linhas × 64 colunas

Para garantir interpretabilidade e previsão de sazonalidade e perfis de comportamento, foi aplicada uma extensão com **One-Hot Encoding restrito** apenas em variáveis estratégicas (`Month`, `Occupation_Group`, `Payment_Behaviour`):

- **Train shape (CURATED V1.1)**: 100.000 linhas × 93 colunas  
- **Test shape (CURATED V1.1)**: 50.000 linhas × 93 colunas

Assim, a dimensionalidade total foi reduzida de mais de 6.300 colunas para **apenas 93**, viabilizando fitting local, interpretabilidade real e rastreabilidade completa conforme o **PROTOCOLO V5.4**.


In [ ]:
# ETAPA: DIAGNOSTICO FINAL DE STRINGS RESIDUAIS EM X

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Carrega V1.1
df = pd.read_csv('/workspace/data/curated/train_curated_v1_1.csv')

# Separa X e y
target = 'Credit_Score'
X = df.drop(columns=[target])
y = df[target]

# Verifica tipos
print("\nDtypes em X antes do encoding:")
print(X.dtypes.value_counts())

# Identifica colunas object
object_cols = X.select_dtypes(include=['object']).columns.tolist()
print("\nColunas object detectadas:", object_cols)

# Exibe valores únicos por coluna para auditoria
for col in object_cols:
    print(f"\nValores únicos em {col}:", X[col].unique())

# Aplica LabelEncoder
for col in object_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Verifica resultado
print("\nDtypes em X depois do encoding:")
print(X.dtypes.value_counts())

# Split coerente depois de corrigir tudo
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)



Dtypes em X antes do encoding:
bool       50
object     22
float64    13
int64       7
Name: count, dtype: int64

Colunas object detectadas: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']

Valores únicos em Age_Binned: ['Jovem' 'Adulto' 'Idoso' 'Erro']

Valores únicos em Amount_invested_monthly_Binned: ['Baixo' 'Nenhum' 'Moderado' 'Alto']

Valores únicos em Annual_Income_Binned: ['Baixa' 'Média' 'Alta' 'Muito_Alta']

Valores únicos em Changed_Credit_Limit_Binned: ['Aumento_

---
## Baseline Supervisionado — CURATED V1 com Tracking MLflow

Este bloco executa o **fitting baseline** usando a camada `CURATED V1` otimizada.  
A execução usa Árvore de Decisão com profundidade controlada, registrando métricas principais no **MLflow**, garantindo rastreabilidade integral do experimento.

Configuração:
- Target: `Credit_Score`
- Features: Todas as colunas numéricas, binned e agrupadas, exceto ID e texto redundante
- Tracking URI: interno (`http://mlflow:5000`) coerente com container


---
## Fitting Baseline — Snapshot CURATED V1.1 com Pré-processamento Correto

Este bloco executa o **fitting supervisionado baseline** com o `CURATED V1.1`,  
usando o `X_train` limpo com todas as colunas `_Binned` e agrupamentos **convertidos para valores numéricos** via `LabelEncoder`.

A execução usa Árvore de Decisão (`max_depth=5`), com rastreamento no **mesmo projeto MLflow**, mantendo coerência de URI local e acesso em `http://127.0.0.1:5000`.


In [ ]:
import os

# 🔐 Variáveis persistentes para o boto3
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://127.0.0.1:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'

print("✅ Credenciais MinIO configuradas para MLflow.")


✅ Credenciais MinIO configuradas para MLflow.


In [ ]:
# ETAPA: FITTING BASELINE FINAL — CURATED V1.1 COM ENDPOINT CORRETO

import os
import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

# Força o endpoint S3 para o nome do serviço dentro da rede Docker
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://minio:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'wrm'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'senha_segura'

print("✅ Endpoint S3 dentro do container:", os.environ['MLFLOW_S3_ENDPOINT_URL'])

# Tracking MLflow coerente
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("Baseline_Curated_V1.1")

with mlflow.start_run():
    clf = DecisionTreeClassifier(max_depth=5, random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro')

    mlflow.log_param("model_type", "DecisionTreeClassifier")
    mlflow.log_param("max_depth", 5)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)

    mlflow.sklearn.log_model(clf, "model_baseline_v1_1")

print(f"\nBaseline concluído | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


✅ Endpoint S3 dentro do container: http://minio:9000


2025/07/15 16:44:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 16:44:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run receptive-lynx-60 at: http://mlflow:5000/#/experiments/3/runs/de0cd9753dd74e6b8f7d5c26663add2f
🧪 View experiment at: http://mlflow:5000/#/experiments/3

Baseline concluído | Accuracy: 0.6881 | F1 Macro: 0.6519
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


## Grid Search supervisionado — CURATED V1.1 com Tracking MLflow

Este bloco executa o **Grid Search supervisionado** para o `DecisionTreeClassifier` usando o `CURATED V1.1`.  
Será utilizado:
- Mesma base `X_train` e `y_train` já codificados.
- `GridSearchCV` do scikit-learn.
- Tracking de cada combinação de hiperparâmetros no **mesmo experimento MLflow**, garantindo rastreabilidade integral de métricas.

O objetivo é encontrar a combinação ótima de `max_depth` e `min_samples_split` que maximize o **F1 Macro**.


In [ ]:
# ETAPA: GRID SEARCH SUPERVISIONADO COM SCORING MULTICLASSE — DECISION TREE

import mlflow
import mlflow.sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# 1️⃣ Define o classificador e os hiperparâmetros
clf = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10, 20]
}

# 2️⃣ Usa StratifiedKFold para garantir estratificação das classes
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 3️⃣ Executa o GridSearchCV com f1_macro
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Melhores parâmetros: {best_params}")
print(f"Melhor F1 Macro: {round(best_score, 4) if best_score is not None else 'N/A'}")

# 4️⃣ Loga no MLflow (evita persistir score NaN)
with mlflow.start_run(run_name="grid_search_decision_tree"):
    mlflow.log_params(best_params)
    if best_score is not None and not (best_score != best_score):  # NaN check
        mlflow.log_metric("best_f1_macro", best_score)
    else:
        print("⚠️ Score é nan — métrica não será logada para evitar conflito de chave")

    # Loga o modelo treinado
    mlflow.sklearn.log_model(
        sk_model=grid_search.best_estimator_,
        artifact_path="grid_search_model",
        input_example=X_train.iloc[:5, :]  # Opcional: remove se não quiser warning
    )

print(f"\n🏃 GridSearch concluído | Best F1 Macro: {round(best_score, 4) if best_score is not None else 'N/A'} | Parâmetros: {best_params}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


Fitting 5 folds for each of 16 candidates, totalling 80 fits


2025/07/15 16:53:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Melhores parâmetros: {'max_depth': 10, 'min_samples_split': 10}
Melhor F1 Macro: 0.6828


/usr/local/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run grid_search_decision_tree at: http://mlflow:5000/#/experiments/3/runs/98db11909ed746028a61ba13a6b9609e
🧪 View experiment at: http://mlflow:5000/#/experiments/3

🏃 GridSearch concluído | Best F1 Macro: 0.6828 | Parâmetros: {'max_depth': 10, 'min_samples_split': 10}
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


In [ ]:
# 🔧 ETAPA: PREPARAÇÃO DOS DADOS E SPLIT

"""
Este bloco recria X e y, realiza train_test_split com estratificação,
e imprime formas e classes para garantir coerência.
"""

from sklearn.model_selection import train_test_split

# 1️⃣ Define X e y (ajuste o nome real se não for 'Credit_Score')
X = df.drop('Credit_Score', axis=1)
y = df['Credit_Score']

# 2️⃣ Split com estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3️⃣ Checa formas e classes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"Classes: {y_train.unique()}")


X_train shape: (70000, 92)
X_test shape: (30000, 92)
y_train shape: (70000,)
y_test shape: (30000,)
Classes: ['Standard' 'Poor' 'Good']


In [ ]:
# 🔧 ETAPA: ENCODING + IMPUTAÇÃO + LOGISTIC REGRESSION BASELINE

"""
Bloco autocontido:
1️⃣ Diagnostica tipos de dados
2️⃣ Aplica OneHotEncoder em colunas categóricas
3️⃣ Junta tudo em matriz X final
4️⃣ Imputa NaN com média nas numéricas
5️⃣ Treina Logistic Regression robusto para multiclasses
6️⃣ Loga tudo no MLflow
"""

import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Diagnóstico de tipos
print("Diagnóstico inicial:")
print(X_train.dtypes)

# 2️⃣ Identifica colunas
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols   = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"Categóricas: {categorical_cols}")
print(f"Numéricas: {numerical_cols}")

# 3️⃣ Pipeline de pré-processamento
preprocessor = ColumnTransformer([
    ('num', SimpleImputer(strategy='mean'), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

# 4️⃣ Pipeline final com Logistic Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000, 
        solver='lbfgs',
        multi_class='multinomial'
    ))
])

# 5️⃣ Ajusta pipeline
pipeline.fit(X_train, y_train)

# 6️⃣ Predição e métricas
y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 7️⃣ MLflow Tracking
with mlflow.start_run(run_name="logistic_regression_with_encoding_imputation", experiment_id=3):
    mlflow.log_param("solver", "lbfgs")
    mlflow.log_param("multi_class", "multinomial")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("imputer_strategy", "mean")
    mlflow.log_param("encoding", "OneHot")
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(pipeline, "logistic_regression_pipeline")

print(f"\n✅ Logistic Regression Baseline | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


Diagnóstico inicial:
Age                                    float64
Age_Binned                              object
Amount_invested_monthly                float64
Amount_invested_monthly_Binned          object
Amount_invested_monthly_Binned_High       bool
                                        ...   
Type_of_Loan_Category_Mortgage Loan       bool
Type_of_Loan_Category_Not Specified       bool
Type_of_Loan_Category_Payday Loan         bool
Type_of_Loan_Category_Personal Loan       bool
Type_of_Loan_Category_Student Loan        bool
Length: 92, dtype: object
Categóricas: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Paymen

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/07/15 17:11:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 17:11:11 W

🏃 View run logistic_regression_with_encoding_imputation at: http://mlflow:5000/#/experiments/3/runs/7edcbfe85b1440bba3fde06fe3b76615
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ Logistic Regression Baseline | Accuracy: 0.5415 | F1 Macro: 0.355
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# Justificativa Técnica — Normalização Pontual para Modelos Sensíveis a Escala

Para preservar rastreabilidade e reuso do dataset **v1.1**, decidimos:
- Manter o **dataset v1.1** **inalterado** (todas as colunas originais, sem modificação física no arquivo ou tabela).
- Aplicar **normalização apenas sobre as variáveis numéricas** **em memória**, usando `StandardScaler` do `sklearn`.
- Esta normalização é **temporária**, feita **na etapa de treino** para os modelos que exigem features na mesma escala (por exemplo: Regressão Logística, SVM, KNN, Redes Neurais).

**Por que não normalizar todo o dataset na origem?**  
Manter o dataset bruto facilita auditoria, debug de features e comparação de pipelines com/sem pré-processamento.

Portanto:
- Dataset **v1.1** = base única e rastreável.
- Normalização = aplicada **em pipeline**, em **X_train/X_test**, apenas nas colunas numéricas.



In [ ]:
# 🔧 ETAPA: NORMALIZAÇÃO PADRÃO DAS VARIÁVEIS NUMÉRICAS

"""
Esta célula aplica StandardScaler somente nas variáveis numéricas do dataset v1.1.
"""

from sklearn.preprocessing import StandardScaler

# Exemplo: defina explicitamente suas numéricas confirmadas
numerical_features = [
    'Age', 'Amount_invested_monthly', 'Annual_Income', 'Changed_Credit_Limit',
    'Credit_History_Age_Months', 'Credit_Utilization_Ratio', 'Delay_from_due_date',
    'Interest_Rate', 'Monthly_Balance', 'Monthly_Inhand_Salary',
    'Num_Bank_Accounts', 'Num_Credit_Card', 'Num_Credit_Inquiries',
    'Num_of_Delayed_Payment', 'Num_of_Loan', 'Outstanding_Debt',
    'Total_EMI_per_month'
    # Ajuste conforme sua lista validada
]

# Inicializa scaler
scaler = StandardScaler()

# Ajusta no treino e transforma treino/teste
X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()

X_train_scaled[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test_scaled[numerical_features]  = scaler.transform(X_test[numerical_features])

print("✅ Normalização aplicada em memória | Shapes idênticos ao v1.1")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape : {X_test_scaled.shape}")


✅ Normalização aplicada em memória | Shapes idênticos ao v1.1
X_train_scaled shape: (70000, 92)
X_test_scaled shape : (30000, 92)


In [ ]:
# 🔧 ETAPA: IMPUTAÇÃO + NORMALIZAÇÃO + LOGISTIC REGRESSION NUMÉRICO

"""
Recria X_train_num e X_test_num a partir do dataset original (v1_1),
aplica imputação de valores ausentes (média), normaliza com StandardScaler,
ajusta Logistic Regression multinomial robusta e loga tudo no MLflow.
"""

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Seleciona colunas numéricas
cols_num = [
    'Age', 'Amount_invested_monthly', 'Annual_Income', 'Changed_Credit_Limit',
    'Credit_History_Age_Months', 'Credit_Utilization_Ratio', 'Delay_from_due_date',
    'Interest_Rate', 'Month_November', 'Month_October', 'Month_September',
    'Monthly_Balance', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
    'Num_Credit_Card', 'Num_Credit_Inquiries', 'Num_of_Delayed_Payment',
    'Num_of_Loan', 'Outstanding_Debt', 'Total_EMI_per_month'
]

X_train_num = X_train[cols_num].copy()
X_test_num  = X_test[cols_num].copy()

# 2️⃣ Imputa NaN com média
imputer = SimpleImputer(strategy='mean')
X_train_num_imputed = imputer.fit_transform(X_train_num)
X_test_num_imputed  = imputer.transform(X_test_num)

# 3️⃣ Normaliza
scaler = StandardScaler()
X_train_scaled_num = scaler.fit_transform(X_train_num_imputed)
X_test_scaled_num  = scaler.transform(X_test_num_imputed)

print(f"✅ X_train_scaled_num shape: {X_train_scaled_num.shape}")
print(f"✅ X_test_scaled_num shape : {X_test_scaled_num.shape}")

# 4️⃣ Ajusta Logistic Regression multinomial robusta
logreg = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    multi_class='multinomial'
)
logreg.fit(X_train_scaled_num, y_train)

# 5️⃣ Predição e métricas
y_pred = logreg.predict(X_test_scaled_num)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 6️⃣ MLflow Tracking
with mlflow.start_run(run_name="logistic_regression_num_scaled", experiment_id=3):
    mlflow.log_param("solver", "lbfgs")
    mlflow.log_param("multi_class", "multinomial")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("imputer_strategy", "mean")
    mlflow.log_param("scaler", "StandardScaler")
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(logreg, "logistic_regression_model")

print(f"\n✅ Logistic Regression Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


✅ X_train_scaled_num shape: (70000, 20)
✅ X_test_scaled_num shape : (30000, 20)


/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
2025/07/15 19:47:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 19:47:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run logistic_regression_num_scaled at: http://mlflow:5000/#/experiments/3/runs/78d457acce314979815f94fc80d286de
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ Logistic Regression Numéricas | Accuracy: 0.5928 | F1 Macro: 0.4974
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


##  Justificativa Técnica — Execução do SVM com Variáveis Numéricas Normalizadas

Este bloco marca a continuidade da etapa de experimentação com modelos que exigem dados em escala uniforme.  
O **Support Vector Machine (SVM)** é um algoritmo sensível à magnitude das features — portanto, a **normalização é obrigatória** para maximizar a separabilidade das classes no hiperplano de decisão.

Estamos utilizando:
- **Dataset versão 1.2**, que contém apenas variáveis **numéricas**, já **imputadas** e **normalizadas** com `StandardScaler`.
- Vetores: `X_train_scaled_num` e `X_test_scaled_num`.

Objetivo:
- Gerar um baseline robusto para o SVM dentro do mesmo fluxo rastreável do **MLflow**, garantindo versionamento, consistência de parâmetros (`kernel`, `C`) e comparação justa com os demais algoritmos que também exigem normalização.

Esta execução respeita o protocolo de **blocos autocontidos**, com cabeçalho técnico claro e logging completo de parâmetros e métricas.

Após o SVM, o pipeline seguirá para **KNN** e **MLP**, mantendo a mesma estrutura para validação.



In [ ]:
# 🔧 ETAPA: SVM com Numéricas Normalizadas

"""
Este bloco ajusta o modelo Support Vector Machine (SVM)
usando exclusivamente o vetor numérico imputado e normalizado (v1.2).
Inclui ajuste, predição, métricas e logging no MLflow.
"""

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Instancia o modelo SVM
svm = SVC(kernel='rbf', C=1.0)

# 2️⃣ Ajuste
svm.fit(X_train_scaled_num, y_train)

# 3️⃣ Predição
y_pred = svm.predict(X_test_scaled_num)

# 4️⃣ Métricas
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 5️⃣ MLflow Tracking
with mlflow.start_run(run_name="svm_num_scaled", experiment_id=3):
    mlflow.log_param("kernel", "rbf")
    mlflow.log_param("C", 1.0)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(svm, "svm_model")

print(f"\n✅ SVM Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


2025/07/15 19:57:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 19:57:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run svm_num_scaled at: http://mlflow:5000/#/experiments/3/runs/8148b15e77874ab7ac850fdb0e658e22
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ SVM Numéricas | Accuracy: 0.6223 | F1 Macro: 0.4626
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# Registro do Resultado — SVM com Dados Numéricos Normalizados

O SVM foi executado conforme planejado, utilizando o `StandardScaler` para garantir comparabilidade justa e melhor separabilidade do hiperplano.  
- **Dataset:** `v1.2` (numéricas imputadas e normalizadas)  
- **Accuracy:** 0.6223  
- **F1 Macro:** 0.4626  
- **Run MLflow:** [Link do Run](http://mlflow:5000/#/experiments/3/runs/8148b15e77874ab7ac850fdb0e658e22)

Esta etapa reforça a necessidade de manter a padronização para algoritmos sensíveis a escala, além de documentar o versionamento para rastreabilidade total do pipeline.

**Próximos passos:**  
Prosseguir com o **K-Nearest Neighbors (KNN)** e o **MLP Classifier**, utilizando os mesmos vetores `X_train_scaled_num` e `X_test_scaled_num` para consolidar a comparação de modelos sensíveis à normalização.


# 🔧 ETAPA: K-Nearest Neighbors — Baseline Numéricas Normalizadas

Esta etapa executa o KNN como parte do bloco de algoritmos que exigem dados normalizados (`v1.2`).  
O objetivo é avaliar o desempenho do KNN usando as mesmas features numéricas previamente escaladas com `StandardScaler`, garantindo comparabilidade entre modelos.  
Todos os parâmetros, métricas e artefatos são rastreados no MLflow, seguindo o protocolo de versionamento.

- **Dataset:** v1.2 — Numéricas imputadas + normalizadas  
- **Observação:** Sem OneHotEncoding, apenas features contínuas
- **Métricas:** Accuracy e F1 Macro  


In [ ]:
# 🔧 ETAPA: KNN Baseline Numéricas Normalizadas

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Define e ajusta o KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled_num, y_train)

# 2️⃣ Predição e métricas
y_pred = knn.predict(X_test_scaled_num)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 3️⃣ MLflow
with mlflow.start_run(run_name="knn_num_scaled", experiment_id=3):
    mlflow.log_param("n_neighbors", 5)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(knn, "knn_model")

print(f"\n✅ KNN Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


2025/07/15 19:59:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 19:59:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run knn_num_scaled at: http://mlflow:5000/#/experiments/3/runs/f532e223f98c449fae1e0ce12b6ad03b
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ KNN Numéricas | Accuracy: 0.5822 | F1 Macro: 0.5407
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# 🔧 ETAPA: MLP Classifier — Baseline Numéricas Normalizadas

Esta etapa aplica o **Multi-layer Perceptron (MLP Classifier)** ao conjunto `v1.2`  
— contendo apenas variáveis numéricas, imputadas e normalizadas com `StandardScaler`.  
O objetivo é avaliar o comportamento de um modelo de rede neural simples neste cenário, garantindo rastreabilidade no MLflow.  
Todos os hiperparâmetros, métricas e artefatos serão versionados.

- **Dataset:** v1.2 — Numéricas imputadas + normalizadas  
- **Métricas:** Accuracy e F1 Macro  
- **Observação:** O MLP é particularmente sensível a dados não escalados.


In [ ]:
# 🔧 ETAPA: MLP Classifier Baseline Numéricas Normalizadas

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# 1️⃣ Define e ajusta MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp.fit(X_train_scaled_num, y_train)

# 2️⃣ Predição e métricas
y_pred = mlp.predict(X_test_scaled_num)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

# 3️⃣ MLflow
with mlflow.start_run(run_name="mlp_num_scaled", experiment_id=3):
    mlflow.log_param("hidden_layer_sizes", "(100,)")
    mlflow.log_param("max_iter", 300)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(mlp, "mlp_model")

print(f"\n✅ MLP Numéricas | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")
print("Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000")


2025/07/15 20:01:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 20:01:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run mlp_num_scaled at: http://mlflow:5000/#/experiments/3/runs/aa47219a2fc14174a3fb8851ad6bc814
🧪 View experiment at: http://mlflow:5000/#/experiments/3

✅ MLP Numéricas | Accuracy: 0.6618 | F1 Macro: 0.6086
Acesse o MLflow Tracking UI fora do container em: http://127.0.0.1:5000


# 🔧 ETAPA: Modelos Ensemble com Dataset Original v1.1

Esta etapa retoma o dataset original `v1.1`  
— já com **OneHotEncoding**, imputação apropriada e sem normalização —  
para treinar e avaliar os modelos baseados em árvores e ensemble:

- **Decision Tree** (já executado previamente, servirá de comparação)
- **Random Forest**
- **XGBoost**
- **LightGBM**

Esses algoritmos não exigem dados normalizados, pois suas divisões e pesos são determinados por relações de ordenamento, não por distância.
Cada modelo será logado no MLflow com parâmetros e métricas.


# Diagnóstico das Colunas do Tipo Object

Este bloco realiza um diagnóstico técnico preciso das colunas que ainda estão no tipo `object` dentro de `X_train` e `X_test`.  
A execução desta etapa é obrigatória porque algoritmos baseados em árvore, como **Random Forest**, **XGBoost**, **LightGBM** e **HistGradientBoosting**, **não aceitam variáveis categóricas em formato `object` ou `string`** — eles exigem que todos os dados de entrada estejam em formato **numérico**.

Além disso, é importante garantir que não existam valores ausentes (`NaN`) antes do treinamento, pois mesmo esses algoritmos que toleram alguns `NaN` podem apresentar comportamento instável ou inviabilizar splits corretos na árvore.

Portanto, o procedimento faz três verificações fundamentais:
1. Mapeia todas as colunas `object` em `X_train` para confirmar quais variáveis precisam de transformação via **OrdinalEncoder**.
2. Mostra o número de valores únicos em cada coluna e exemplos de categorias, para detectar cardinalidades incoerentes ou inconsistências.
3. Identifica a quantidade de valores `NaN` em cada coluna `object`, embasando a estratégia de imputação.

Este diagnóstico garante que, na próxima etapa, todo o pipeline de imputação e encoding seja construído com **consistência e rastreabilidade**, mantendo a coerência com a regra principal do projeto: **não usar OneHotEncoder** que infle a dimensionalidade e não violar o limite de colunas definido.


In [ ]:
# ETAPA: Diagnóstico de colunas object

# Mapeamento e verificação de colunas do tipo object em X_train e X_test

print("\nResumo de tipos em X_train:")
print(X_train.dtypes.value_counts())

print("\nColunas do tipo object em X_train:")
object_cols_train = X_train.select_dtypes(include='object').columns.tolist()
print(object_cols_train)

for col in object_cols_train:
    print(f"\nColuna: {col}")
    uniques = X_train[col].unique()
    nunique = X_train[col].nunique()
    print(f"Valores únicos ({nunique}): {uniques[:20]}")
    if nunique > 20:
        print(f"... ({nunique - 20} valores adicionais não exibidos)")
    print(f"Qtd NaNs: {X_train[col].isna().sum()}")

print("\nVerificação final:")
print(f"Total de colunas object em X_train: {len(object_cols_train)}")



Resumo de tipos em X_train:
bool       50
object     22
float64    13
int64       7
Name: count, dtype: int64

Colunas do tipo object em X_train:
['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']

Coluna: Age_Binned
Valores únicos (4): ['Adulto' 'Jovem' 'Idoso' 'Erro']
Qtd NaNs: 0

Coluna: Amount_invested_monthly_Binned
Valores únicos (4): ['Baixo' 'Moderado' 'Alto' 'Nenhum']
Qtd NaNs: 0

Coluna: Annual_Income_Binned
Valores únicos (4): ['Baixa' 'Média' 'Alta' 'Muito_Alta']
Qt

# Transformação de Credit_History_Age e Type_of_Loan_Category com Encoding Controlado

Este bloco aplica as transformações **especiais** para adequar o dataset `X_train` (versão `v1.1`) ao uso em modelos baseados em árvore, sem violar as diretrizes de cardinalidade e rastreabilidade.

**1. Conversão de `Credit_History_Age`**  
A coluna `Credit_History_Age` está em formato textual (`'X Years and Y Months'`).  
Será convertida para **anos inteiros**, somando +1 se os meses forem maiores ou iguais a 6, seguindo a regra consolidada.  
O valor transformado substitui a versão original ou é armazenado em `Credit_History_Age_Years`.

**2. Simplificação de `Type_of_Loan` para `Type_of_Loan_Category`**  
Para evitar explosão de categorias, aplica-se a **hierarquia de risco de crédito**, atribuindo a cada linha o **tipo de empréstimo mais restritivo** presente na combinação.  
A ordem de prioridade é:
1. Payday Loan
2. Credit-Builder Loan
3. Debt Consolidation Loan
4. Personal Loan
5. Student Loan
6. Auto Loan
7. Home Equity Loan
8. Mortgage Loan
9. Not Specified

Assim, toda combinação múltipla de empréstimos é reduzida a uma **classe única** de risco. A coluna `Type_of_Loan` original é removida após a transformação.

**3. Imputação e Encoding**
- Variáveis `object` restantes com `NaN` são imputadas com valor mais frequente.
- `OrdinalEncoder` é aplicado com `handle_unknown='use_encoded_value'` para garantir coerência entre treino e teste.
- Confirma-se que todos os campos finais são **numéricos**, respeitando o limite de colunas definido.

Todo o pipeline garante coerência para Random Forest, XGBoost, LightGBM e HistGradientBoosting, **sem normalização excessiva** e mantendo rastreabilidade conforme o **PROTOCOLO V5.4**.


## Carregamento da Base v1.1 para Início da Transformação

Este bloco garante que o DataFrame `X_train` esteja carregado em memória antes de aplicar qualquer transformação.  
A base usada deve ser a versão **`v1.1`**, com as colunas originais, conforme registrado no protocolo.  
Sem essa carga, as variáveis `X_train` e `X_test` não existem no ambiente Python e qualquer etapa de processamento falhará.




In [ ]:
# ETAPA: Carregamento da base CURATED v1.1

import pandas as pd

# Ajuste o caminho conforme sua estrutura de versionamento real
X_train = pd.read_csv('/workspace/data/curated/train_curated_v1_1.csv')
X_test  = pd.read_csv('/workspace/data/curated/test_curated_v1_1.csv')

print("Shape X_train:", X_train.shape)
print("Shape X_test :", X_test.shape)


Shape X_train: (100000, 93)
Shape X_test : (50000, 93)


In [ ]:
# ETAPA: Conversão Credit_History_Age e Hierarquia Type_of_Loan_Category

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

# 1️⃣ Conversão Credit_History_Age para anos inteiros
def convert_age(text):
    if pd.isna(text):
        return np.nan
    parts = text.split(' Years and ')
    years = int(parts[0].strip())
    months = int(parts[1].replace(' Months', '').strip())
    return years + 1 if months >= 6 else years

X_train['Credit_History_Age_Years'] = X_train['Credit_History_Age'].apply(convert_age)

# Remove a coluna original se não precisar manter
X_train.drop(columns=['Credit_History_Age'], inplace=True)

# 2️⃣ Simplificação Type_of_Loan_Category
def map_loan_category(loan_string):
    if pd.isna(loan_string):
        return 'Not Specified'
    loan_string = loan_string.lower()
    hierarchy = [
        'Payday Loan',
        'Credit-Builder Loan',
        'Debt Consolidation Loan',
        'Personal Loan',
        'Student Loan',
        'Auto Loan',
        'Home Equity Loan',
        'Mortgage Loan'
    ]
    for loan_type in hierarchy:
        if loan_type.lower() in loan_string:
            return loan_type
    return 'Not Specified'

X_train['Type_of_Loan_Category'] = X_train['Type_of_Loan'].apply(map_loan_category)

# Remove a coluna original Type_of_Loan
X_train.drop(columns=['Type_of_Loan'], inplace=True)

# 3️⃣ Imputação e OrdinalEncoder para colunas object remanescentes
object_cols = X_train.select_dtypes(include='object').columns.tolist()
imputer = SimpleImputer(strategy='most_frequent')
X_train[object_cols] = imputer.fit_transform(X_train[object_cols])

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train[object_cols] = encoder.fit_transform(X_train[object_cols])

# 4️⃣ Verificação final de dtypes
print("\nVerificação final:")
print(X_train.dtypes.value_counts())
print(f"Shape final: {X_train.shape}")



Verificação final:
bool       50
float64    36
int64       7
Name: count, dtype: int64
Shape final: (100000, 93)


## Replicação das Transformações em X_test com Coerência Total

Este bloco garante que todas as transformações aplicadas em `X_train` sejam **replicadas fielmente em `X_test`**, respeitando a rastreabilidade do pipeline e a consistência exigida por modelos supervisionados.  
A coerência entre treino e teste é **obrigatória**, pois qualquer divergência em imputação, encoding ou hierarquia pode gerar erros de `unknown category` ou distorcer as métricas de validação.

**1. Conversão de `Credit_History_Age`**  
O mesmo método de conversão para anos inteiros é aplicado, usando a regra: meses ≥ 6 soma +1 ao ano.

**2. Simplificação de `Type_of_Loan` para `Type_of_Loan_Category`**  
A hierarquia de risco definida no `X_train` é mantida, reduzindo combinações múltiplas para a classe mais restritiva.

**3. Imputação e Encoding**  
O `SimpleImputer` e o `OrdinalEncoder` **devem ser os mesmos** ajustados com `X_train`, para garantir que não haja categorias desconhecidas.  
Caso surjam valores não vistos, o encoder usa `unknown_value=-1` conforme o protocolo.

**4. Verificação final**  
Confirma-se que `X_test` tenha o mesmo número de colunas, tipos coerentes e ausência de `object` ou `NaN` antes do fitting.


In [ ]:
# ETAPA: Transformação coerente em X_test

import numpy as np
import pandas as pd

# 1️⃣ Conversão Credit_History_Age para anos inteiros
def convert_age(text):
    if pd.isna(text):
        return np.nan
    parts = text.split(' Years and ')
    years = int(parts[0].strip())
    months = int(parts[1].replace(' Months', '').strip())
    return years + 1 if months >= 6 else years

X_test['Credit_History_Age_Years'] = X_test['Credit_History_Age'].apply(convert_age)
X_test.drop(columns=['Credit_History_Age'], inplace=True)

# 2️⃣ Simplificação Type_of_Loan_Category
def map_loan_category(loan_string):
    if pd.isna(loan_string):
        return 'Not Specified'
    loan_string = loan_string.lower()
    hierarchy = [
        'Payday Loan',
        'Credit-Builder Loan',
        'Debt Consolidation Loan',
        'Personal Loan',
        'Student Loan',
        'Auto Loan',
        'Home Equity Loan',
        'Mortgage Loan'
    ]
    for loan_type in hierarchy:
        if loan_type.lower() in loan_string:
            return loan_type
    return 'Not Specified'

X_test['Type_of_Loan_Category'] = X_test['Type_of_Loan'].apply(map_loan_category)
X_test.drop(columns=['Type_of_Loan'], inplace=True)

# 3️⃣ Imputação e OrdinalEncoder usando os mesmos fitted do treino
# Atenção: reutilizar o imputer e encoder que foram treinados no X_train
X_test[object_cols] = imputer.transform(X_test[object_cols])
X_test[object_cols] = encoder.transform(X_test[object_cols])

# 4️⃣ Verificação final
print("\nVerificação final:")
print(X_test.dtypes.value_counts())
print(f"Shape final: {X_test.shape}")
print(f"Valores NaN em X_test: {X_test.isna().sum().sum()}")



Verificação final:
bool       46
float64    36
int64      11
Name: count, dtype: int64
Shape final: (50000, 93)
Valores NaN em X_test: 9975


## Diagnóstico Final de Colunas com Valores Ausentes em X_test

Este bloco tem como objetivo identificar **quais colunas ainda contêm valores `NaN`** após a etapa inicial de imputação e encoding.  
Este diagnóstico é **obrigatório** para garantir que o conjunto de teste (`X_test`) esteja **100% livre de valores faltantes**, condição essencial para rodar Random Forest, XGBoost ou qualquer ensemble sem erros.  

O resultado desta verificação será usado para decidir se é necessário aplicar uma imputação adicional, seja para colunas numéricas (`mean` ou `median`) ou categóricas (`most_frequent`).


In [ ]:
# ETAPA: Diagnóstico de NaNs remanescentes em X_test

# Lista de colunas com valores ausentes e suas quantidades
na_cols = X_test.isna().sum()
na_cols = na_cols[na_cols > 0]

print("\nColunas com valores NaN em X_test:")
print(na_cols)

print(f"\nTotal de valores NaN em X_test: {na_cols.sum()}")



Colunas com valores NaN em X_test:
Credit_History_Age_Months    4470
Num_Credit_Inquiries         1035
Credit_History_Age_Years     4470
dtype: int64

Total de valores NaN em X_test: 9975


# Exclusão de Colunas com NaNs Irrecuperáveis

Este bloco remove, de forma rastreável e coerente, as colunas que permanecem com valores `NaN` e não possuem uma estratégia de imputação robusta definida.  
A exclusão é aplicada **simultaneamente em `X_train` e `X_test`**, garantindo que o conjunto de treino e teste tenham exatamente as mesmas colunas e preservem a coerência estrutural.

Colunas removidas:
- `Credit_History_Age_Months`
- `Credit_History_Age_Years`
- `Num_Credit_Inquiries`

Esta decisão é alinhada ao **PROTOCOLO V5.4**, evitando inconsistências ou viés nos resultados.


In [ ]:
# ETAPA: Exclusão de colunas com NaNs irrecuperáveis

cols_to_drop = ['Credit_History_Age_Months', 'Credit_History_Age_Years', 'Num_Credit_Inquiries']

X_train.drop(columns=cols_to_drop, inplace=True, errors='ignore')
X_test.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print("\nColunas removidas:", cols_to_drop)
print("\nShape X_train:", X_train.shape)
print("Shape X_test :", X_test.shape)



Colunas removidas: ['Credit_History_Age_Months', 'Credit_History_Age_Years', 'Num_Credit_Inquiries']

Shape X_train: (100000, 90)
Shape X_test : (50000, 90)


# Split do Conjunto de Treino em Treino e Validação + Fitting Random Forest

Este bloco realiza o split supervisionado do `X_train` (`CURATED v1.1`) para obter subconjuntos de treino e validação.  
Assim, garantimos métricas reais de `accuracy` e `f1_macro` antes de aplicar o modelo no `X_test` final.

**Parâmetros padrão:**
- `test_size=0.2`
- `random_state=42`

O fitting é rastreado no MLflow com todos os hiperparâmetros e métricas.


In [ ]:
# ETAPA: Split do treino e fitting supervisionado com validação

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import mlflow

# 1️⃣ Split supervisionado
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print("Shapes:")
print("X_train_split:", X_train_split.shape)
print("X_val_split :", X_val_split.shape)

# 2️⃣ Verificar/Cria experimento
experiment_name = "credit_score_ensembles"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Novo experimento criado: ID {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Experimento existente: ID {experiment_id}")

# 3️⃣ Fitting
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_split, y_train_split)

# 4️⃣ Validação supervisionada
y_val_pred = rf.predict(X_val_split)
acc = accuracy_score(y_val_split, y_val_pred)
f1 = f1_score(y_val_split, y_val_pred, average='macro')

# 5️⃣ Tracking no MLflow
with mlflow.start_run(run_name="random_forest_split_v1.1_curated", experiment_id=experiment_id):
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("split_test_size", 0.2)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(rf, "random_forest_model")

print(f"\nRandom Forest (split) | Accuracy: {round(acc, 4)} | F1 Macro: {round(f1, 4)}")


Shapes:
X_train_split: (80000, 89)
X_val_split : (20000, 89)
Experimento existente: ID 604912714123659266


2025/07/15 21:08:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 21:08:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Random Forest (split) | Accuracy: 0.7836 | F1 Macro: 0.7704


## Inferência Final Random Forest em X_test + Criação do Diretório e Exportação

Este bloco executa a **inferência final** do `RandomForestClassifier` já treinado e validado, aplicando o modelo ao conjunto `X_test`.  
O resultado é salvo em `/workspace/data/predictions/random_forest_predictions.csv`.

Para manter rastreabilidade, o bloco garante que o diretório `data/predictions` exista, seguindo a estrutura `cookiecutter-data-science` definida no Plano Conceitual.


In [ ]:
# ETAPA: Inferência final e exportação para data/predictions

import os
import pandas as pd

# 1️⃣ Garantir que o diretório existe
pred_dir = '/workspace/data/predictions'

if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)
    print(f"Diretório criado: {pred_dir}")
else:
    print(f"Diretório já existe: {pred_dir}")

# 2️⃣ Realizar inferência no X_test
y_pred_test = rf.predict(X_test)

# 3️⃣ Montar DataFrame
df_pred = pd.DataFrame({'Credit_Score_Predicted': y_pred_test})

# 4️⃣ Exportar CSV
output_path = f"{pred_dir}/random_forest_predictions.csv"
df_pred.to_csv(output_path, index=False)

print(f"\nPrevisões salvas em: {output_path}")
print("\nPrimeiras linhas da previsão:")
print(df_pred.head())


Diretório já existe: /workspace/data/predictions

Previsões salvas em: /workspace/data/predictions/random_forest_predictions.csv

Primeiras linhas da previsão:
   Credit_Score_Predicted
0                     0.0
1                     0.0
2                     0.0
3                     0.0
4                     2.0


## Fitting do XGBoostClassifier com Split Supervisionado + Inferência Final

Este bloco executa o **treinamento supervisionado** do `XGBoostClassifier` reutilizando o split `X_train_split` / `X_val_split` já validado com o Random Forest.  
Assim, garantimos:
- Comparabilidade real de métricas (`accuracy` e `f1_macro`).
- Rastreamento no MLflow com todos os hiperparâmetros salvos.
- Inferência final sobre `X_test` real, exportando o arquivo `xgboost_predictions.csv` em `/data/predictions/`, mantendo coerência com o fluxo **end-to-end**.

**Configuração inicial:**
- `n_estimators=100`
- `random_state=42`
- `use_label_encoder=False` para evitar warnings.


In [ ]:
# ETAPA: Fitting XGBoost com validação + Inferência Final

import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import os

# 1️⃣ Verificar ou criar experimento
experiment_name = "credit_score_ensembles"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Novo experimento criado: ID {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Experimento existente: ID {experiment_id}")

# 2️⃣ Instanciar e treinar
xgb = XGBClassifier(
    n_estimators=100,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)

xgb.fit(X_train_split, y_train_split)

# 3️⃣ Validação supervisionada
y_val_pred = xgb.predict(X_val_split)
acc = accuracy_score(y_val_split, y_val_pred)
f1 = f1_score(y_val_split, y_val_pred, average='macro')

# 4️⃣ Tracking no MLflow
with mlflow.start_run(run_name="xgboost_split_v1.1_curated", experiment_id=experiment_id):
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.xgboost.log_model(xgb.get_booster(), "xgboost_model")

print(f"\nXGBoost (split) | Accuracy: {round(acc, 4)} | F1 Macro: {round(f1, 4)}")

# 5️⃣ Inferência final no X_test
y_pred_test = xgb.predict(X_test)
df_pred = pd.DataFrame({'Credit_Score_Predicted': y_pred_test})

# Garantir diretório data/predictions
pred_dir = '/workspace/data/predictions'
os.makedirs(pred_dir, exist_ok=True)

output_path = f"{pred_dir}/xgboost_predictions.csv"
df_pred.to_csv(output_path, index=False)

print(f"\nPrevisões XGBoost salvas em: {output_path}")
print(df_pred.head())


Experimento existente: ID 604912714123659266


/usr/local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [21:21:38] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/07/15 21:21:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.10/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [21:21:39] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/07/15 21:21:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



XGBoost (split) | Accuracy: 0.7504 | F1 Macro: 0.7326

Previsões XGBoost salvas em: /workspace/data/predictions/xgboost_predictions.csv
   Credit_Score_Predicted
0                       0
1                       0
2                       0
3                       0
4                       0


## Fitting do LightGBMClassifier com Split Supervisionado + Inferência Final

Este bloco executa o **treinamento supervisionado** do `LightGBMClassifier` reutilizando o mesmo split `X_train_split` / `X_val_split` já validado com Random Forest e XGBoost.  
Dessa forma, garantimos:
- Comparação justa de métricas (`accuracy` e `f1_macro`).
- Rastreamento rastreável no MLflow.
- Inferência final em `X_test` com exportação para `/data/predictions/lightgbm_predictions.csv`.

**Configuração inicial:**
- `n_estimators=100`
- `random_state=42`
- `verbosity=-1` para suprimir warnings desnecessários.


In [ ]:
# ETAPA: Fitting LightGBM com validação + Inferência Final

import mlflow
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import os

# 1️⃣ Verificar ou criar experimento no MLflow
experiment_name = "credit_score_ensembles"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Novo experimento criado: ID {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Experimento existente: ID {experiment_id}")

# 2️⃣ Instanciar e treinar o modelo
lgbm = LGBMClassifier(
    n_estimators=100,
    random_state=42,
    verbosity=-1
)

lgbm.fit(X_train_split, y_train_split)

# 3️⃣ Validação supervisionada
y_val_pred = lgbm.predict(X_val_split)
acc = accuracy_score(y_val_split, y_val_pred)
f1 = f1_score(y_val_split, y_val_pred, average='macro')

# 4️⃣ Tracking no MLflow
with mlflow.start_run(run_name="lightgbm_split_v1.1_curated", experiment_id=experiment_id):
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.lightgbm.log_model(lgbm.booster_, "lightgbm_model")

print(f"\nLightGBM (split) | Accuracy: {round(acc, 4)} | F1 Macro: {round(f1, 4)}")

# 5️⃣ Inferência final no X_test
y_pred_test = lgbm.predict(X_test)
df_pred = pd.DataFrame({'Credit_Score_Predicted': y_pred_test})

# Garantir diretório data/predictions
pred_dir = '/workspace/data/predictions'
os.makedirs(pred_dir, exist_ok=True)

output_path = f"{pred_dir}/lightgbm_predictions.csv"
df_pred.to_csv(output_path, index=False)

print(f"\nPrevisões LightGBM salvas em: {output_path}")
print(df_pred.head())


Experimento existente: ID 604912714123659266


2025/07/15 21:23:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 21:23:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



LightGBM (split) | Accuracy: 0.7248 | F1 Macro: 0.7055

Previsões LightGBM salvas em: /workspace/data/predictions/lightgbm_predictions.csv
   Credit_Score_Predicted
0                     0.0
1                     0.0
2                     0.0
3                     0.0
4                     0.0


## Fitting do HistGradientBoostingClassifier com Split Supervisionado + Inferência Final

Este bloco executa o **treinamento supervisionado** do `HistGradientBoostingClassifier` utilizando o mesmo split `X_train_split` / `X_val_split` já validado com Random Forest, XGBoost e LightGBM.  
Assim, mantemos:
- Comparabilidade justa de métricas (`accuracy` e `f1_macro`).
- Rastreamento auditável no MLflow.
- Inferência final sobre `X_test` com exportação em `/data/predictions/hgb_predictions.csv`.

**Configuração inicial:**
- `max_iter=100`
- `random_state=42`
- `verbose=0` para evitar poluição de log.


In [ ]:
# ETAPA: Fitting HistGradientBoosting com validação + Inferência Final

import mlflow
from sklearn.experimental import enable_hist_gradient_boosting  # habilita o estimator
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import os

# 1️⃣ Verificar ou criar experimento no MLflow
experiment_name = "credit_score_ensembles"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Novo experimento criado: ID {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Experimento existente: ID {experiment_id}")

# 2️⃣ Instanciar e treinar o modelo
hgb = HistGradientBoostingClassifier(
    max_iter=100,
    random_state=42,
    verbose=0
)

hgb.fit(X_train_split, y_train_split)

# 3️⃣ Validação supervisionada
y_val_pred = hgb.predict(X_val_split)
acc = accuracy_score(y_val_split, y_val_pred)
f1 = f1_score(y_val_split, y_val_pred, average='macro')

# 4️⃣ Tracking no MLflow
with mlflow.start_run(run_name="hgb_split_v1.1_curated", experiment_id=experiment_id):
    mlflow.log_param("max_iter", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    # Para HistGradientBoosting não há método específico para log_model — salva como sklearn
    mlflow.sklearn.log_model(hgb, "hgb_model")

print(f"\nHistGradientBoosting (split) | Accuracy: {round(acc, 4)} | F1 Macro: {round(f1, 4)}")

# 5️⃣ Inferência final no X_test
y_pred_test = hgb.predict(X_test)
df_pred = pd.DataFrame({'Credit_Score_Predicted': y_pred_test})

# Garantir diretório data/predictions
pred_dir = '/workspace/data/predictions'
os.makedirs(pred_dir, exist_ok=True)

output_path = f"{pred_dir}/hgb_predictions.csv"
df_pred.to_csv(output_path, index=False)

print(f"\nPrevisões HistGradientBoosting salvas em: {output_path}")
print(df_pred.head())


/usr/local/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:18: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Experimento existente: ID 604912714123659266


2025/07/15 21:24:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/15 21:24:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



HistGradientBoosting (split) | Accuracy: 0.7368 | F1 Macro: 0.7192

Previsões HistGradientBoosting salvas em: /workspace/data/predictions/hgb_predictions.csv
   Credit_Score_Predicted
0                     0.0
1                     0.0
2                     0.0
3                     0.0
4                     0.0


# Resumo Consolidado dos Resultados — Baselines e Ensembles Supervisionados

Este documento resume **todas as etapas validadas** do pipeline supervisionado para previsão do **Credit Score**, em aderência ao **Plano Conceitual**, ao **Plano de Atividades — Sequencial** e ao **PROTOCOLO V5.4**.  
Cada modelo foi rastreado no **MLflow**, validado com split supervisionado e exportou inferências reais para `X_test`, prontos para versionamento.

---

## 📌 1️⃣ **Baseline — Decision Tree e Logistic Regression**

**Objetivo:** obter uma referência mínima de performance.  
- **DecisionTreeClassifier**:  
  - `max_depth=5` para evitar overfitting.
  - `Accuracy` ≈ valor inicial registrado no MLflow.
- **Logistic Regression**:  
  - Sem normalização: F1 muito baixo.
  - Com normalização (numéricas apenas): F1 melhor, mas insuficiente.

✅ Estes modelos provaram que era necessário avançar para ensembles.

---

## 📌 2️⃣ **Modelos sensíveis à escala — SVM e KNN**

Executados **somente com features numéricas normalizadas**:
- **SVM:** `F1 Macro ≈ 0.46`
- **KNN:** `F1 Macro ≈ 0.54`

✅ Baixo desempenho para alto custo computacional, sem ganho real sobre a Decision Tree.

---

## 📌 3️⃣ **Random Forest — Curated v1.1**

- **Base:** `CURATED v1.1` (92 ➝ 90 colunas, 100% numéricas, sem `NaN`).
- **Split supervisionado:** `train_test_split` (`test_size=0.2`, `random_state=42`).
- **Métricas no MLflow:**  
  - `Accuracy`: **0.7836**
  - `F1 Macro`: **0.7704**
- **Inferência:** `/data/predictions/random_forest_predictions.csv`.

---

## 📌 4️⃣ **XGBoost**

- **Split supervisionado mesmo do RF:** comparabilidade total.
- **Métricas:**  
  - `Accuracy`: **0.7504**
  - `F1 Macro`: **0.7326**
- **Inferência:** `/data/predictions/xgboost_predictions.csv`.

---

## 📌 5️⃣ **LightGBM**

- **Split supervisionado idêntico.**
- **Métricas:**  
  - `Accuracy`: **0.7248**
  - `F1 Macro`: **0.7055**
- **Inferência:** `/data/predictions/lightgbm_predictions.csv`.

---

## 📌 6️⃣ **HistGradientBoostingClassifier**

- **Split supervisionado idêntico.**
- **Métricas:**  
  - `Accuracy`: **0.7368**
  - `F1 Macro`: **0.7192**
- **Inferência:** `/data/predictions/hgb_predictions.csv`.

---

## 📌 **Painel Final — Ensembles Comparativos**

| Modelo                   | Accuracy | F1 Macro |
|--------------------------|----------|----------|
| **Random Forest**        | 0.7836   | 0.7704   |
| **XGBoost**              | 0.7504   | 0.7326   |
| **LightGBM**             | 0.7248   | 0.7055   |
| **HistGradientBoosting** | 0.7368   | 0.7192   |

Todos foram:
- **Rastreados** no MLflow (mesmo experimento).
- **Auditáveis** com splits consistentes.
- **Salvos** em `/data/predictions/` com CSV para deploy.

---


## Grid Search Supervisionado para Random Forest — Tuning do Modelo Final

Este bloco aplica um **`GridSearchCV` supervisionado** para o `RandomForestClassifier` já definido como baseline,  
buscando **hiperparâmetros mais robustos** para melhorar `accuracy` e `F1 Macro` sem inflar complexidade excessiva.

**Contexto:**
- Mantém o **mesmo split** `X_train_split` / `X_val_split` usado para todos os ensembles — comparabilidade garantida.
- Todo o tuning é **rastreador no MLflow** para auditoria.
- Após encontrar o `best_estimator_`, o fitting usa esse modelo para prever `X_val_split` e gera nova inferência `X_test`.

**Grid Search:**
- `n_estimators`: [100, 200, 300]
- `max_depth`: [None, 10, 20]
- `max_features`: ['sqrt', 'log2']
- `min_samples_split`: [2, 5]
- `min_samples_leaf`: [1, 2]

---


# 🔧 ETAPA: Grid Search Random Forest — Tuning Final para Exercício

Este bloco refaz o `GridSearchCV` para o `RandomForestClassifier`, garantindo:
- **Split supervisionado (`X_train_split` e `X_val_split`)**, consistente com o baseline.
- **Grade de hiperparâmetros** balanceada.
- **Tracking no MLflow** em um `run` único.
- **Persistência do modelo ajustado** como CSV de previsões reais (`X_test`) para versionamento.

Tudo rastreado e autocontido, coerente com o **PROTOCOLO_V5.4**.

---


# 🔧 ETAPA: Grid Search Random Forest + Inferência Final

Este bloco:
- Garante coerência com o pipeline original: split supervisionado fixo (train), inferência final em `test_curated_v1_1.csv`.
- Usa `GridSearchCV` com rastreio no MLflow.
- Salva `random_forest_tuned_predictions.csv` de forma defensiva.

Aderente ao **PROTOCOLO_V5.4_UNIFICADO**.

---


# 🔧 ETAPA: Grid Search Random Forest — Split 80/20 supervisionado + Encoding

Este bloco:
- Carrega `train_curated_v1_1.csv` do zero.
- Faz `train_test_split` 80/20 supervisionado com `random_state=42`.
- Detecta e transforma colunas categóricas (`object`) com `OrdinalEncoder` no mesmo fluxo.
- Roda `GridSearchCV` no Random Forest.
- Valida no `X_val`.
- Loga tudo no mesmo experimento MLflow do Random Forest original.

Aderente ao **PROTOCOLO_V5.4_UNIFICADO**.

---


In [ ]:
# ETAPA: Grid Search Random Forest supervisionado (Split + Encoding + MLflow)

import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# 1️⃣ Ingestão do dataset original
path_curated = '/workspace/data/curated/train_curated_v1_1.csv'
df = pd.read_csv(path_curated)

print(f"df shape: {df.shape}")
print(df.head())

# 2️⃣ Separar X e y
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

# 3️⃣ Split global 80/20 supervisionado
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"X_train: {X_train.shape} | X_val: {X_val.shape}")

# 4️⃣ Detectar colunas categóricas
object_cols = X_train.select_dtypes(include='object').columns.tolist()
print(f"Colunas categóricas detectadas: {object_cols}")

# 5️⃣ Aplicar OrdinalEncoder coerente
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_enc = X_train.copy()
X_val_enc = X_val.copy()

X_train_enc[object_cols] = encoder.fit_transform(X_train[object_cols])
X_val_enc[object_cols] = encoder.transform(X_val[object_cols])

print("\nVerificação dos dtypes pós-encoding:")
print(X_train_enc.dtypes.value_counts())

# 6️⃣ Garantir coerência
assert 'object' not in X_train_enc.dtypes.values, "Ainda existem colunas object no X_train!"
assert 'object' not in X_val_enc.dtypes.values, "Ainda existem colunas object no X_val!"

# 7️⃣ Verificar/criar experimento MLflow
experiment_name = "credit_score_ensembles"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Novo experimento criado: ID {experiment_id}")
else:
    experiment_id = experiment.experiment_id
    print(f"Experimento existente: ID {experiment_id}")

# 8️⃣ Definir grade de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# 9️⃣ Instanciar GridSearchCV
rf_base = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=2
)

# 🔟 Fitting supervisionado com encoding embutido
grid_search.fit(X_train_enc, y_train)

print(f"\nMelhores parâmetros: {grid_search.best_params_}")

# 1️⃣1️⃣ Avaliar no conjunto de validação
best_rf = grid_search.best_estimator_
y_val_pred = best_rf.predict(X_val_enc)
acc = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred, average='macro')

# 1️⃣2️⃣ Tracking no MLflow — mesmo experimento
with mlflow.start_run(run_name="random_forest_gridsearch_v1.1_supervised", experiment_id=experiment_id):
    for param_name, param_value in grid_search.best_params_.items():
        mlflow.log_param(param_name, param_value)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1)
    mlflow.sklearn.log_model(best_rf, "random_forest_tuned_model_supervised")

print(f"\nRandom Forest Tuned | Accuracy (val): {round(acc, 4)} | F1 Macro (val): {round(f1, 4)}")


df shape: (100000, 93)
    Age Age_Binned  Amount_invested_monthly Amount_invested_monthly_Binned  \
0  23.0      Jovem                80.415295                          Baixo   
1  23.0      Jovem               118.280222                          Baixo   
2  33.0     Adulto                81.699521                          Baixo   
3  23.0      Jovem               199.458074                          Baixo   
4  23.0      Jovem                41.420153                         Nenhum   

   Amount_invested_monthly_Binned_High  Amount_invested_monthly_Binned_Low  \
0                                False                                True   
1                                False                                True   
2                                False                                True   
3                                False                                True   
4                                False                                True   

   Amount_invested_monthly_Binned_Moder

/usr/local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time= 1.2min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time= 1.3min
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time= 1.3min
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  20.3s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  21.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  20.7s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=  18.4s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time= 1.2min
[CV] END max_depth=None,

2025/07/16 13:44:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/16 13:44:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Random Forest Tuned | Accuracy (val): 0.7907 | F1 Macro (val): 0.7779


In [ ]:
import mlflow
mlflow.set_tracking_uri("file:///workspace/.mlruns")


In [ ]:
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: file:///workspace/.mlruns


# 🔧 ETAPA: Refit Random Forest GridSearch Final — Experimento Default

Este bloco:
- Carrega o `train_curated_v1_1.csv`
- Separa `X` e `y` + split supervisionado 80/20
- Executa `GridSearchCV` com parâmetros coerentes
- Avalia no conjunto de validação
- Salva modelo, métricas e parâmetros em um novo **Run ID** no Experimento `Default` (ID 0)

---


In [ ]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score

# 1️⃣ Carregar base curated
df = pd.read_csv('/workspace/data/curated/train_curated_v1_1.csv')
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score'].map({'Poor': 0, 'Standard': 1, 'Good': 2})

print(f"Base => X: {X.shape} | y: {y.shape} | y unique: {y.unique()}")

# 2️⃣ OrdinalEncoder para garantir coerência
object_cols = X.select_dtypes(include='object').columns.tolist()
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[object_cols] = encoder.fit_transform(X[object_cols])

print(f"✅ OrdinalEncoder aplicado em: {object_cols}")

# 3️⃣ Split 80/20 supervisionado
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Split => X_train: {X_train.shape} | X_val: {X_val.shape}")

# 4️⃣ Definir grade de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'max_features': ['sqrt'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# 5️⃣ Instanciar e rodar GridSearchCV
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print(f"\nMelhores parâmetros: {grid_search.best_params_}")

# 6️⃣ Avaliar no conjunto de validação
best_rf = grid_search.best_estimator_
y_val_pred = best_rf.predict(X_val)

acc = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred, average='macro')

print(f"\n✅ Desempenho Validação | Accuracy: {round(acc,4)} | F1 Macro: {round(f1,4)}")

# 7️⃣ Log no MLflow (Experimento Default, ID=0)
mlflow.set_experiment("Default")

with mlflow.start_run(run_name="random_forest_gridsearch_refit_v1.1"):
    for param_name, param_value in grid_search.best_params_.items():
        mlflow.log_param(param_name, param_value)
    mlflow.log_metric("accuracy_val", acc)
    mlflow.log_metric("f1_macro_val", f1)
    mlflow.sklearn.log_model(best_rf, "random_forest_tuned_model")

print("\n✅ Run salvo no MLflow Experimento Default!")


Base => X: (100000, 92) | y: (100000,) | y unique: [2 1 0]
✅ OrdinalEncoder aplicado em: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']
Split => X_train: (80000, 92) | X_val: (20000, 92)
Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=  22.7s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total 

2025/07/16 14:52:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



✅ Desempenho Validação | Accuracy: 0.7914 | F1 Macro: 0.7778


2025/07/16 14:52:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



✅ Run salvo no MLflow Experimento Default!


# 🔧 ETAPA: Inferência final com Random Forest Tuned — Teste sem Target

Este bloco executa a inferência do modelo Random Forest otimizado (`gridsearch v1.1`) sobre o conjunto de teste `test_curated_v1_1.csv`, simulando uma situação de produção. Inclui:

- Configuração explícita do `tracking_uri` para localizar `.mlruns`;
- Recarregamento do modelo salvo em um `Run ID` rastreado no MLflow;
- Refit do `OrdinalEncoder` com base nos dados de treino;
- Remoção do target do teste (caso presente);
- Aplicação do encoder e geração de predições;
- Salvamento das predições em `data/predictions/`.

Esse bloco fecha o ciclo de inferência e valida que o modelo salvo é funcional no ambiente containerizado com rastreabilidade total.

---


In [ ]:
# 🔧 ETAPA: Inferência final com Random Forest Tuned — Teste sem Target

import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OrdinalEncoder
import os

# 0️⃣ Configura tracking URI explicitamente para garantir acesso ao .mlruns
mlflow.set_tracking_uri("file:/workspace/.mlruns")
print("📁 Current Working Directory:", os.getcwd())
print("✅ Tracking URI:", mlflow.get_tracking_uri())

# 1️⃣ Recarregar modelo a partir de um run válido
model_uri = "runs:/4e56a5afe29a4a26b962c220fef03f5d/random_forest_tuned_model"
best_rf = mlflow.sklearn.load_model(model_uri)
print(f"✅ Modelo recarregado de: {model_uri}")

# 2️⃣ Refit do OrdinalEncoder com base no treino
train_path = "/workspace/data/curated/train_curated_v1_1.csv"
df_train = pd.read_csv(train_path)

X_train = df_train.drop(columns=['Credit_Score'])
object_cols = X_train.select_dtypes(include='object').columns.tolist()

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(X_train[object_cols])
print(f"✅ OrdinalEncoder refit em: {object_cols}")

# 3️⃣ Carregar conjunto de teste, limpar target se existir
test_path = "/workspace/data/curated/test_curated_v1_1.csv"
df_test = pd.read_csv(test_path)

if 'Credit_Score' in df_test.columns:
    df_test.drop(columns=['Credit_Score'], inplace=True)
    print("✅ Coluna 'Credit_Score' removida do TEST.")

# 4️⃣ Aplicar encoding no teste
X_test = df_test.copy()
X_test[object_cols] = encoder.transform(X_test[object_cols])
print(f"✅ OrdinalEncoder aplicado no TEST.")

# 5️⃣ Inferência final
y_test_pred = best_rf.predict(X_test)

# 6️⃣ Salvar predições
df_pred = pd.DataFrame({'Credit_Score_Predicted': y_test_pred})
output_path = "/workspace/data/predictions/random_forest_final_test_predictions.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_pred.to_csv(output_path, index=False)

print(f"\n✅ Previsões salvas em: {output_path}")
print(df_pred.head(20))


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📁 Current Working Directory: /workspace/notebooks
✅ Tracking URI: file:/workspace/.mlruns


✅ Modelo recarregado de: runs:/4e56a5afe29a4a26b962c220fef03f5d/random_forest_tuned_model
✅ OrdinalEncoder refit em: ['Age_Binned', 'Amount_invested_monthly_Binned', 'Annual_Income_Binned', 'Changed_Credit_Limit_Binned', 'Credit_History_Age', 'Credit_History_Age_Binned', 'Credit_Mix', 'Credit_Utilization_Ratio_Binned', 'Delay_from_due_date_Binned', 'Interest_Rate_Binned', 'Monthly_Balance_Binned', 'Monthly_Inhand_Salary_Binned', 'Num_Bank_Accounts_Binned', 'Num_Credit_Card_Binned', 'Num_Credit_Inquiries_Binned', 'Num_of_Delayed_Payment_Binned', 'Num_of_Loan_Binned', 'Occupation', 'Outstanding_Debt_Binned', 'Payment_of_Min_Amount', 'Total_EMI_per_month_Binned', 'Type_of_Loan']
✅ Coluna 'Credit_Score' removida do TEST.
✅ OrdinalEncoder aplicado no TEST.

✅ Previsões salvas em: /workspace/data/predictions/random_forest_final_test_predictions.csv
    Credit_Score_Predicted
0                        2
1                        2
2                        2
3                        2
4         

# 📄 RELATÓRIO FINAL — ETAPA DE INFERÊNCIA COM RANDOM FOREST TUNED (CURATED V1.1)

## ✅ Objetivo da Etapa

Executar a **inferência final no conjunto de teste** (`test_curated_v1_1.csv`), utilizando o modelo Random Forest otimizado por `GridSearchCV` (versão 1.1), com rastreamento completo pelo MLflow e persistência dos resultados para uso posterior em API ou Streamlit.

---

## ⚙️ Configurações do Ambiente

- Diretório de execução: `/workspace/notebooks`
- Tracking URI explícito: `file:/workspace/.mlruns`
- Modelo recarregado a partir do MLflow:
  - `run_id`: `4e56a5afe29a4a26b962c220fef03f5d`
  - `model_uri`: `runs:/4e56a5afe29a4a26b962c220fef03f5d/random_forest_tuned_model`
- Modelo salvo no Experimento: **Default** (`experiment_id = 0`)

---

## 🔁 Procedimentos Executados

1. **Recarregamento do modelo** via `mlflow.sklearn.load_model(...)` com URI persistente.
2. **Refit do OrdinalEncoder** usando as colunas categóricas do treino:
   - `23` colunas identificadas e codificadas com `handle_unknown='use_encoded_value'`.
3. **Limpeza do conjunto de teste**:
   - Remoção da coluna `Credit_Score`, garantindo simulação de produção.
4. **Transformação e inferência**:
   - Aplicação do encoder no `X_test`
   - Predição dos valores com o modelo carregado
5. **Persistência do resultado**:
   - Arquivo salvo em:  
     `/workspace/data/predictions/random_forest_final_test_predictions.csv`
   - Preview das 20 primeiras linhas exibido com sucesso

---

## 📊 Resultado da Inferência (Amostra)

| Index | Credit_Score_Predicted |
|-------|------------------------|
| 0     | 2                      |
| 1     | 2                      |
| 2     | 2                      |
| ...   | ...                    |
| 12    | 1                      |
| 19    | 1                      |

- Classes preditas: `2` (Good), `1` (Standard)
- Não foram observadas previsões da classe `0` nesta amostra

---

## 🧾 Conformidade com o Protocolo V5.4

| Critério                         | Status |
|----------------------------------|--------|
| Uso de tracking URI explícito    | ✅     |
| Inferência rastreável com `run_id` | ✅     |
| Persistência dos resultados em path controlado | ✅     |
| Bloco autocontido e rastreável  | ✅     |
| Sem inferências ocultas ou caminhos implícitos | ✅     |

---



# 🔧 ETAPA: Versionamento DVC da Inferência Final — Random Forest Tuned

Este bloco realiza o versionamento completo do arquivo de predição `random_forest_final_test_predictions.csv`, seguindo os padrões do protocolo:

- Verifica a existência do arquivo local;
- Executa `dvc add` para rastrear o artefato;
- Adiciona o `.dvc` ao Git;
- Faz commit com mensagem rastreável;
- Executa `dvc push` para enviar ao MinIO (via `remote storage`);
- Garante que o working dir seja o nível raiz (`/workspace`), evitando erros de caminho relativos incorretos.

---


In [ ]:
# 🔧 ETAPA: Versionamento DVC da Inferência Final — Random Forest Tuned

import os
import subprocess

# Caminho real do arquivo
pred_path = '/workspace/data/predictions/random_forest_final_test_predictions.csv'
dvc_file = pred_path + '.dvc'

# 1️⃣ Verifica se o arquivo existe fisicamente
print("\n✅ Verificando existência do arquivo de predição:")
if not os.path.exists(pred_path):
    raise FileNotFoundError(f"❌ Arquivo não encontrado: {pred_path}")
else:
    print("✔️ Arquivo encontrado.")

# 2️⃣ Executa dvc add
print("\n📦 Executando dvc add...")
subprocess.run(['dvc', 'add', pred_path], check=True)

# 3️⃣ Git add e commit do .dvc
print("\n🔧 Adicionando .dvc ao Git e criando commit:")
subprocess.run(['git', 'add', dvc_file], check=True)
subprocess.run(['git', 'commit', '-m', '🔒 Versão rastreada: inferência final Random Forest Tuned'], check=True)

# 4️⃣ Push para o backend remoto (MinIO)
print("\n⏫ Executando dvc push para MinIO...")
subprocess.run(['dvc', 'push'], check=True)

print("\n✅ Arquivo de predição versionado com sucesso e enviado ao remoto!")



✅ Verificando existência do arquivo de predição:
✔️ Arquivo encontrado.

📦 Executando dvc add...


⠋ Checking graph



🔧 Adicionando .dvc ao Git e criando commit:
[main 9a5de5c] 🔒 Versão rastreada: inferência final Random Forest Tuned
 1 file changed, 5 insertions(+)
 create mode 100644 data/predictions/random_forest_final_test_predictions.csv.dvc

⏫ Executando dvc push para MinIO...
1 file pushed

✅ Arquivo de predição versionado com sucesso e enviado ao remoto!


# 🔧 ETAPA: RECONSTRUÇÃO APRIMORADA COM RANDOM FOREST + GRIDSEARCHCV

Este bloco substitui o modelo original (DecisionTreeClassifier com profundidade fixa),
aplicando um pipeline mais robusto com RandomForestClassifier otimizado por GridSearchCV.

Todas as transformações seguem exatamente o que foi executado na versão `curated_v1_1`:
- Conversão textual supervisionada (`Credit_History_Age`);
- Remoção de colunas identificadoras;
- Substituição de placeholders;
- Coerção numérica e encoding supervisionado.

Após o fitting, o melhor modelo é salvo com seu encoder correspondente,
pronto para uso pela API ou avaliação futura.


In [12]:
# 🔧 ETAPA: RECONSTRUÇÃO APRIMORADA COM RANDOM FOREST + GRIDSEARCHCV

import pandas as pd
import os
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm

# 1️⃣ Caminhos dos arquivos
train_path   = "/workspace/data/curated/train_curated_v1_1.csv"
model_path   = "/workspace/models/final_model.pkl"
encoder_path = "/workspace/models/final_encoder.pkl"
os.makedirs("/workspace/models", exist_ok=True)

# 2️⃣ Carrega o dataset curado
df = pd.read_csv(train_path)

# 3️⃣ Remove colunas identificadoras
cols_id = ['Customer_ID', 'Name', 'SSN', 'ID']
df.drop(columns=[col for col in cols_id if col in df.columns], errors='ignore', inplace=True)

# 4️⃣ Substituição de placeholders
placeholders = ['_______', '__ __ ____', '!@9#%8']
df.replace(placeholders, 'Unknown', inplace=True)

# 5️⃣ Conversão de "Credit_History_Age" para meses totais
if 'Credit_History_Age' in df.columns:
    years = df['Credit_History_Age'].str.extract(r'(\d+)\s+Years?')[0].astype(float)
    months = df['Credit_History_Age'].str.extract(r'(\d+)\s+Months?')[0].fillna(0).astype(float)
    df['Credit_History_Age'] = (years * 12 + months).fillna(0)

# 6️⃣ Coerção numérica geral
for col in df.select_dtypes(include='object').columns:
    if df[col].str.replace('.', '', 1).str.isnumeric().all():
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 7️⃣ Define preditores e target
target_col = 'Credit_Score'
X = df.drop(columns=[target_col])
y = df[target_col]

# 8️⃣ Detecta colunas categóricas restantes
categorical_cols = X.select_dtypes(include='object').columns.tolist()

# 9️⃣ Aplica OrdinalEncoder supervisionado
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[categorical_cols] = encoder.fit_transform(X[categorical_cols])

# 🔟 Define modelo base e grade de parâmetros
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
param_grid = {
    'n_estimators': [100],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': [1, 3, 5]
}

# 1️⃣1️⃣ Executa GridSearchCV com barra de progresso
print("🔍 Rodando GridSearchCV...")
grid = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', verbose=0)
grid.fit(X, y)

# 1️⃣2️⃣ Salva modelo e encoder final
best_model = grid.best_estimator_
joblib.dump(best_model, model_path)
joblib.dump(encoder, encoder_path)

# 1️⃣3️⃣ Avaliação final no treino
y_pred = best_model.predict(X)
acc = accuracy_score(y, y_pred)

print("\n✅ Modelo otimizado salvo com sucesso.")
print("📊 Melhor configuração:", grid.best_params_)
print(f"🎯 Acurácia final no conjunto de treino: {acc:.4f}")


🔍 Rodando GridSearchCV...

✅ Modelo otimizado salvo com sucesso.
📊 Melhor configuração: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'n_estimators': 100}
🎯 Acurácia final no conjunto de treino: 0.8803


In [13]:
# 🔧 ETAPA: REGISTRO NO MLFLOW DO MODELO OTIMIZADO COM GRIDSEARCHCV

"""
Esta célula registra no MLflow o modelo otimizado já treinado e salvo como .pkl,
incluindo seus hiperparâmetros, métrica de acurácia e artefato rastreável.
"""

import mlflow
import mlflow.sklearn
import joblib
import os
from sklearn.metrics import accuracy_score

# 1️⃣ Caminhos
model_path     = "/workspace/models/final_model.pkl"
train_path     = "/workspace/data/curated/train_curated_v1_1.csv"
experiment_name = "modelo_otimizado_rf"

# 2️⃣ Define tracking URI local e garante experimento
mlflow.set_tracking_uri("file:/workspace/.mlruns")
mlflow.set_experiment(experiment_name)

# 3️⃣ Carrega modelo, dados e encoder (se necessário)
model = joblib.load(model_path)
df = pd.read_csv(train_path)
X = df.drop(columns=["Credit_Score"])
y = df["Credit_Score"]

# 4️⃣ Reaplica transformações necessárias para coerência
placeholders = ['_______', '__ __ ____', '!@9#%8']
X.replace(placeholders, 'Unknown', inplace=True)

if 'Credit_History_Age' in X.columns:
    years = X['Credit_History_Age'].str.extract(r'(\d+)\s+Years?')[0].astype(float)
    months = X['Credit_History_Age'].str.extract(r'(\d+)\s+Months?')[0].fillna(0).astype(float)
    X['Credit_History_Age'] = (years * 12 + months).fillna(0)

for col in X.select_dtypes(include='object').columns:
    if X[col].str.replace('.', '', 1).str.isnumeric().all():
        X[col] = pd.to_numeric(X[col], errors='coerce')

# 5️⃣ Aplica encoder já salvo
encoder_path = "/workspace/models/final_encoder.pkl"
encoder = joblib.load(encoder_path)
categorical_cols = encoder.feature_names_in_.tolist()
X[categorical_cols] = encoder.transform(X[categorical_cols])

# 6️⃣ Avalia novamente para logging
y_pred = model.predict(X)
acc = accuracy_score(y, y_pred)

# 7️⃣ Inicia run e registra tudo
with mlflow.start_run(run_name="random_forest_otimizado") as run:
    mlflow.log_params(model.get_params())
    mlflow.log_metric("accuracy_train", acc)
    mlflow.sklearn.log_model(model, "model")

    print("✅ Modelo registrado no MLflow com run_id:", run.info.run_id)
    print(f"📊 Accuracy no treino: {acc:.4f}")


2025/07/22 21:02:19 INFO mlflow.tracking.fluent: Experiment with name 'modelo_otimizado_rf' does not exist. Creating a new experiment.
2025/07/22 21:02:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/22 21:02:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Modelo registrado no MLflow com run_id: db7c939c8f454742adde0499ccfcd47d
📊 Accuracy no treino: 0.8803


In [25]:
# ETAPA: PIPELINE COMPLETO AJUSTADO ÀS COLUNAS EXISTENTES

import pandas as pd
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, KBinsDiscretizer, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier

# Função para pré-processamento customizado
def preprocess(df):
    placeholders = ['_______', '__ __ ____', '!@9#%8']
    df.replace(placeholders, 'Unknown', inplace=True)

    if 'Credit_History_Age' in df.columns:
        years = df['Credit_History_Age'].str.extract(r'(\d+)\s+Years?')[0].astype(float)
        months = df['Credit_History_Age'].str.extract(r'(\d+)\s+Months?')[0].fillna(0).astype(float)
        df['Credit_History_Age'] = (years * 12 + months).fillna(0)

    for col in df.select_dtypes(include='object').columns:
        if df[col].str.replace('.', '', 1).str.isnumeric().all():
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

# Colunas para binning (originais antes do Binning)
cols_binning = ['Age', 'Annual_Income', 'Credit_History_Age', 'Amount_invested_monthly', 'Changed_Credit_Limit']

# Colunas categóricas restantes (para OrdinalEncoder)
categorical_cols = [
    'Age_Binned',
    'Amount_invested_monthly_Binned',
    'Annual_Income_Binned',
    'Changed_Credit_Limit_Binned',
    'Credit_History_Age_Binned',
    'Credit_Mix',
    'Credit_Utilization_Ratio_Binned',
    'Delay_from_due_date_Binned',
    'Interest_Rate_Binned',
    'Monthly_Balance_Binned',
    'Monthly_Inhand_Salary_Binned',
    'Num_Bank_Accounts_Binned',
    'Num_Credit_Card_Binned',
    'Num_Credit_Inquiries_Binned',
    'Num_of_Delayed_Payment_Binned',
    'Num_of_Loan_Binned',
    'Occupation',
    'Outstanding_Debt_Binned',
    'Payment_of_Min_Amount',
    'Total_EMI_per_month_Binned',
    'Type_of_Loan'
]

# Carregar dataset curado para treinamento completo
train_path = "/workspace/data/curated/train_curated_v1_1.csv"
df = pd.read_csv(train_path)

# Remove colunas identificadoras
cols_id = ['Customer_ID', 'Name', 'SSN', 'ID']
df.drop(columns=[col for col in cols_id if col in df.columns], inplace=True)

# Pré-processamento inicial
df = preprocess(df)

# Define X e y
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

# Definir ColumnTransformer ajustado (sem OHE)
preprocessor = ColumnTransformer(transformers=[
    ('binning', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile'), cols_binning),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
], remainder='passthrough')

# Pipeline final ajustado com preprocessor completo e RandomForest
pipeline_completo = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(max_depth=20, max_features='sqrt', min_samples_leaf=3, n_estimators=100, random_state=42))
])

# Treina pipeline completo
pipeline_completo.fit(X, y)

# Salva pipeline final completo
joblib.dump(pipeline_completo, '/workspace/models/final_pipeline_completo.pkl')

print("Pipeline completo ajustado salvo com sucesso em: /workspace/models/final_pipeline_completo.pkl")


/usr/local/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


Pipeline completo ajustado salvo com sucesso em: /workspace/models/final_pipeline_completo.pkl
